# Advent of Code 2024

Let's try out [ocaml](https://ocaml.org/) this year!  I read a book about Standard ML like 20 years ago, but remember very little.  So there will be a learning curve, but it shouldn't be as bad as [the Advent of APL](https://github.com/jeredw/advent2021/blob/main/Advent%20of%20Code%202021.ipynb).

Is this thing on?

In [1]:
2+2;;

- : int = 4


## Table of Contents

| Su | Mo | Tu | We | Th | Fr | Sa |
| -- | -- | -- | -- | -- | -- | -- |
| [1](#day1) | [2](#day2) | [3](#day3) | [4](#day4) | [5](#day5) | [6](#day6) | [7](#day7) |
| [8](#day8) | [9](#day9) | [10](#day10) | [11](#day11) | [12](#day12) | [13](#day13) | [14](#day14) |
| [15](#day15) | [16](#day16) | 17 | 18 | 19 | 20 | 21 |
| 22 | 23 | 24 | 25 | 26 | 27 | 28 |
| 29 | 30 | 31 |    |    |    |    |


## Reusable utilities

This section is for stuff I get tired of copying and pasting.

### Ok, what is "core"?

On Day 3, while looking for basic I/O utilities again (is there really no "read entire file" library?), I learned there's an entire alternative standard library written by Jane Street.  This feels weird, but whatever.

In [2]:
#use "topfind" ;;

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()


In [3]:
#require "core" ;;

/Users/jered/.opam/4.12.0/lib/base/base_internalhash_types: added to search path
/Users/jered/.opam/4.12.0/lib/base/base_internalhash_types/base_internalhash_types.cma: loaded
/Users/jered/.opam/4.12.0/lib/base/caml: added to search path
/Users/jered/.opam/4.12.0/lib/base/caml/caml.cma: loaded
/Users/jered/.opam/4.12.0/lib/base/shadow_stdlib: added to search path
/Users/jered/.opam/4.12.0/lib/base/shadow_stdlib/shadow_stdlib.cma: loaded
/Users/jered/.opam/4.12.0/lib/sexplib0: added to search path
/Users/jered/.opam/4.12.0/lib/sexplib0/sexplib0.cma: loaded
/Users/jered/.opam/4.12.0/lib/base: added to search path
/Users/jered/.opam/4.12.0/lib/base/base.cma: loaded
/Users/jered/.opam/4.12.0/lib/fieldslib: added to search path
/Users/jered/.opam/4.12.0/lib/fieldslib/fieldslib.cma: loaded
/Users/jered/.opam/4.12.0/lib/ppx_compare/runtime-lib: added to search path
/Users/jered/.opam/4.12.0/lib/ppx_compare/runtime-lib/ppx_compare_lib.cma: loaded
/Users/jered/.opam/4.12.0/lib/ppx_enumerate/run

In [4]:
open Core

### Oh God, what have I done

Apparently this is an overlay library, and just blows away much of the core language.  Now my `List.map` function is backwards!  That's... incredibly antisocial, actually.  I guess what did I expect from a quantitative finance company.

In [5]:
List.map ((+) 0) [1;2;3]

File "[5]", line 1, characters 9-16:
1 | List.map ((+) 0) [1;2;3]
             ^^^^^^^
Warning 5 [ignored-partial-application]: this function application is partial,
maybe some arguments are missing.


error: compile_error

In [6]:
List.map [1;2;3] ((+) 0)

- : int list = [1; 2; 3]


Hang on, let me just rewrite everything now.

In [7]:
open Core.List

Apparently I need this too.  Have I mentioned it's impossible to Google search for punctuation?  Ugh.

### Reading stuff from files

We seem to have to read stuff from files a lot.

In [8]:
(* Core has a way to do this *)
In_channel.read_lines "day1_example.txt"

- : Base.string Base.list =
["3   4"; "4   3"; "2   5"; "1   3"; "3   9"; "3   3"]


Now there are different combinators, too.  How do these work?

In [9]:
[1;2;3] >>| ((+) 1)

- : int list = [2; 3; 4]


In [10]:
(*let read_numbers_from_file file_name =
  let file = open_in file_name in
  let output = ref [] in
  try
    while true do
      let line = input_line file
               |> Str.split (Str.regexp " +")
               |> List.map int_of_string in
      output := line :: !output
    done;
    assert false
  with
    End_of_file -> List.rev !output *)

let read_numbers_from_file file_name =
  let parse line =
    line |> Str.split (Str.regexp " +") >>| int_of_string in
  In_channel.read_lines file_name >>| parse

val read_numbers_from_file : Base.string -> int list list = <fun>


### This keeps coming up

In [11]:
let pair_of_list xs =
  match xs with
  | [x; y] -> (x, y)
  | _ -> failwith "expecting pair"

val pair_of_list : 'a list -> 'a * 'a = <fun>


### Reading grids

In [12]:
let read_grid_from_file file_name =
  let lines = In_channel.read_lines file_name in
  let rows = length lines in
  let cols = String.length (nth_exn lines 0) in
  let grid = Array.make_matrix rows cols '.' in
  for i = 0 to rows-1 do
    for j = 0 to cols-1 do
      grid.(i).(j) <- String.get (nth_exn lines i) j
    done
  done;
  grid

val read_grid_from_file : Base.string -> char Core.Array.t Core.Array.t =
  <fun>


### Cartesian product without the diagonal

In [13]:
let rec pairs xs = 
  match xs with
  | [] -> []
  | [x] -> []
  | x::rest -> (rest>>|(fun y -> (x,y))) @ pairs rest

val pairs : 'a list -> ('a * 'a) Base.List.t = <fun>


In [14]:
pairs [1;2;3;4;5;6]

- : (int * int) Base.List.t =
[(1, 2); (1, 3); (1, 4); (1, 5); (1, 6); (2, 3); (2, 4); (2, 5); (2, 6);
 (3, 4); (3, 5); (3, 6); (4, 5); (4, 6); (5, 6)]


## <a name=day1>Day 1</a>

[Link](https://adventofcode.com/2024/day/1)

### Part 1

They want me to read two columns from a file, sort them, and sum the differences.

In [15]:
let read_pairs_from_file file_name =
  read_numbers_from_file file_name
  >>| pair_of_list

val read_pairs_from_file : Base.string -> (int * int) list = <fun>


In [16]:
read_pairs_from_file "day1_example.txt"

- : (int * int) list = [(3, 4); (4, 3); (2, 5); (1, 3); (3, 9); (3, 3)]


Ok, that was a little ugly, but it could be worse.  Let's use some of this great type matching stuff.

In [17]:
let rec unzip pairs =
  match pairs with
  | [] -> ([], [])
  | (x, y) :: rest ->
    let (xs, ys) = unzip rest in
      (x::xs, y::ys)

val unzip : ('a * 'b) list -> 'a list * 'b list = <fun>


In [18]:
unzip([(1,2); (3,4); (5,6)])

- : int list * int list = ([1; 3; 5], [2; 4; 6])


That was fun.  ~But there's a library for this.~  Update: Ok, core seems to actually call this `zip` and `unzip`...

In [19]:
List.unzip [(1,2); (3,4); (5,6)]

- : int list * int list = ([1; 3; 5], [2; 4; 6])


In [20]:
zip [1;3;5] [2;4;6]

- : (int * int) list Core.List.Or_unequal_lengths.t =
Core.List.Or_unequal_lengths.Ok [(1, 2); (3, 4); (5, 6)]


Oh no, I overwrote `compare` when I did `open Core.List` to try to get the list combinators.  What a mess.

In [21]:
compare

- : ('a -> 'a -> int) -> 'a list -> 'a list -> int = <fun>


In [22]:
Stdlib.compare

- : 'a -> 'a -> int = <fun>


Hmm, post `zip`ping, my list type is infected with `Core.List.Or_unequal_lengths.t` and I can't do unsanctioned list stuff with it.  I guess I have to use `zip_exn` to fail hard.

In [23]:
let day1_part1 pairs =
  let (xs, ys) = unzip pairs in
  (zip_exn (sort xs Stdlib.compare) (sort ys Stdlib.compare)
    >>| fun (x, y) -> abs(x - y))
  |> List.fold ~f:(+) ~init:0

val day1_part1 : (int * int) list -> int = <fun>


~That is actually kind of pretty, although the syntax is meh.~  Let's be real, this is an ugly way to write a simple program.

In [24]:
read_pairs_from_file "day1_example.txt" |> day1_part1

- : int = 11


In [25]:
read_pairs_from_file "day1_input.txt" |> day1_part1

- : int = 1666427


### Part 2

Now we've got to count stuff.  There is no need to be especially clever about this.

In [26]:
let day1_part2 pairs =
  let (xs, ys) = unzip pairs in
  let count x = List.count ys ((=) x) in
    xs
    >>| (fun x -> x * (count x))
    |> List.fold ~f:(+) ~init:0

val day1_part2 : (Core_kernel.Int.t * Core_kernel.Int.t) list -> int = <fun>


In [27]:
read_pairs_from_file "day1_input.txt" |> day1_part2

- : int = 24316233


## <a name="day2">Day 2</a>

[Link](https://adventofcode.com/2024/day/2)

### Part 1

Today we have to take adjacent differences of things.

In [28]:
let rec differences list =
  match list with
  | [] -> []
  | [x] -> [] 
  | x :: y :: rest -> x - y :: differences (y :: rest)

val differences : int list -> int list = <fun>


In [29]:
differences [7; 6; 4; 2; 1]

- : int list = [1; 2; 2; 1]


I kind of wish this were point free?  Like I wanted to write `increasing || decreasing`.  Oh well.

In [30]:
let safe levels =
  let decreasing = List.for_all ~f:(fun(x) -> x < 0) in
  let increasing = List.for_all ~f:(fun(x) -> x > 0) in
  let in_range = List.for_all ~f:(fun(x) -> abs(x) >= 1 && abs(x) <= 3) in
  let check xs = (increasing xs || decreasing xs) && in_range xs in
    check (differences levels)

let count safety_check data =
  data
  |> List.filter ~f:safety_check
  |> List.length

val safe : int list -> bool = <fun>


val count : ('a -> bool) -> 'a list -> int = <fun>


In [31]:
read_numbers_from_file "day2_example.txt" |> count safe

- : int = 2


In [32]:
read_numbers_from_file "day2_input.txt" |> count safe

- : int = 442


### Part 2

Now we have to try fudging the data, I guess.

In [33]:
let rec ablate xs =
  match xs with
  | [] -> []
  | x :: rest -> rest :: (List.map ~f:(fun(xs) -> x :: xs)) (ablate rest)

val ablate : 'a list -> 'a list list = <fun>


In [34]:
ablate [1;2;3;4]

- : int list list = [[2; 3; 4]; [1; 3; 4]; [1; 2; 4]; [1; 2; 3]]


In [35]:
let safeish levels =
  safe levels ||
  (ablate levels |> List.exists ~f:safe)

val safeish : int list -> bool = <fun>


In [36]:
read_numbers_from_file "day2_example.txt" |> count safeish

- : int = 4


In [37]:
read_numbers_from_file "day2_input.txt" |> count safeish

- : int = 493


## <a name="day3">Day 3</a>

### Part 1

Today we are parsing instructions from strings full of junk.  There is a builtin regex library from, like, the 90s.  Since we're already doing this core stuff, I guess we may as well just use re2 and unlock the _sheer power_ of NFAs.  Stand back.

In [38]:
#require "re2" ;;

/Users/jered/.opam/4.12.0/lib/core_kernel/rope: added to search path
/Users/jered/.opam/4.12.0/lib/core_kernel/rope/rope.cma: loaded
/Users/jered/.opam/4.12.0/lib/re2/c: added to search path
/Users/jered/.opam/4.12.0/lib/re2/c/re2_c.cma: loaded
/Users/jered/.opam/4.12.0/lib/re2: added to search path
/Users/jered/.opam/4.12.0/lib/re2/re2.cma: loaded


In [39]:
let find_day3_instructions s =
  let mul = Re2.create_exn {|mul\((\d+,\d+)\)|} in
  Re2.find_all_exn ~sub:(`Index 1) mul s
  >>| fun (xs) -> (String.split ~on:',' xs
                   >>| int_of_string
                   |> pair_of_list)

val find_day3_instructions : string -> (int * int) list = <fun>


In [40]:
find_day3_instructions "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"

- : (int * int) list = [(2, 4); (5, 5); (11, 8); (8, 5)]


In [41]:
let day3_part1 s =
  find_day3_instructions s
  >>| (fun ((a, b)) -> a * b)
   |> fold ~init:0 ~f:(+)

val day3_part1 : string -> int = <fun>


In [42]:
In_channel.read_all "day3_example.txt" |> day3_part1

- : int = 161


In [43]:
In_channel.read_all "day3_input.txt" |> day3_part1

- : int = 170778545


### Part 2

Oh good, now we get to parse more instructions - we can turn multiplying on and off, how convenient.  Let's use a type thing.

In [44]:
type day3_instruction =
  Day3Mul of int * int
  | Day3Do
  | Day3Dont

type day3_instruction = Day3Mul of int * int | Day3Do | Day3Dont


In [45]:
let parse_day3_program s =
  let opcode = Re2.create_exn {|(don't\(\)|do\(\)|mul\(\d+,\d+\))|} in
  let parse_opcode args =
    match args with
    | "mul" :: x :: y :: rest -> Day3Mul (int_of_string x, int_of_string y)
    | "do" :: rest -> Day3Do
    | "don't" :: rest -> Day3Dont
    | _ -> failwith "unknown instruction" in
  Re2.find_all_exn ~sub:(`Index 1) opcode s
  >>| String.split_on_chars ~on:[','; '('; ')']
  >>| parse_opcode

val parse_day3_program : string -> day3_instruction list = <fun>


In [46]:
parse_day3_program "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"

- : day3_instruction list =
[Day3Mul (2, 4); Day3Dont; Day3Mul (5, 5); Day3Mul (11, 8); Day3Do;
 Day3Mul (8, 5)]


Let's shake things up and evaluate this imperatively.  Mwahaha, purists, take that.

In [47]:
let eval_day3_program s =
  let mul_enabled = ref true in
  let eval instruction =
    match instruction with
    | Day3Do -> mul_enabled := true; 0
    | Day3Dont -> mul_enabled := false; 0
    | Day3Mul (x, y) -> if !mul_enabled then x * y else 0 in
  parse_day3_program s
  >>| eval
  |> fold ~init:0 ~f:(+)

val eval_day3_program : string -> int = <fun>


In [48]:
eval_day3_program "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"

- : int = 48


In [49]:
In_channel.read_all "day3_input.txt" |> eval_day3_program

- : int = 82868252


## <a name="day4">Day 4</a>

### Part 1

Today we are doing word searches, which I guess means we need arrays.  Let's see how this works.

In [50]:
let read_wordsearch file_name = read_grid_from_file file_name

val read_wordsearch : Base.string -> char Core.Array.t Core.Array.t = <fun>


In [51]:
read_wordsearch "day4_example.txt"

- : char Core.Array.t Core.Array.t =
[|[|'M'; 'M'; 'M'; 'S'; 'X'; 'X'; 'M'; 'A'; 'S'; 'M'|];
  [|'M'; 'S'; 'A'; 'M'; 'X'; 'M'; 'S'; 'M'; 'S'; 'A'|];
  [|'A'; 'M'; 'X'; 'S'; 'X'; 'M'; 'A'; 'A'; 'M'; 'M'|];
  [|'M'; 'S'; 'A'; 'M'; 'A'; 'S'; 'M'; 'S'; 'M'; 'X'|];
  [|'X'; 'M'; 'A'; 'S'; 'A'; 'M'; 'X'; 'A'; 'M'; 'M'|];
  [|'X'; 'X'; 'A'; 'M'; 'M'; 'X'; 'X'; 'A'; 'M'; 'A'|];
  [|'S'; 'M'; 'S'; 'M'; 'S'; 'A'; 'S'; 'X'; 'S'; 'S'|];
  [|'S'; 'A'; 'X'; 'A'; 'M'; 'A'; 'S'; 'A'; 'A'; 'A'|];
  [|'M'; 'A'; 'M'; 'M'; 'M'; 'X'; 'M'; 'M'; 'M'; 'M'|];
  [|'M'; 'X'; 'M'; 'X'; 'A'; 'X'; 'M'; 'A'; 'S'; 'X'|]|]


In [52]:
let count_matches word grid =
  let count_matches_at i j di dj =
    let len = String.length word in
    let found = ref 1 in
    for n = 0 to len-1 do
      try
        let word_char = String.get word n in
        let grid_char = grid.(i + di * n).(j + dj * n) in
          if Char.(<>) word_char grid_char then found := 0
      with
        Invalid_argument _ -> found := 0
    done; !found in
  let matches = ref 0 in
    let rows = Array.length grid in
    let cols = Array.length grid.(0) in
    let dirs = [(-1, -1); (0, -1); (1, -1);
                (-1,  0);          (1,  0);
                (-1,  1); (0,  1); (1,  1)] in
    for i = 0 to rows-1 do
      for j = 0 to cols-1 do
        let matches_here = dirs
          >>| (fun((di, dj)) -> count_matches_at i j di dj)
          |> fold ~init:0 ~f:(+) in
          matches := !matches + matches_here
      done
    done; !matches

val count_matches : string -> Core.Char.t Core.Array.t Core.Array.t -> int =
  <fun>


Why doesn't the inequality operator `<>` work right for characters?  That's weird.  This Jane Street library is banana pancakes.

In [53]:
read_wordsearch "day4_example.txt" |> count_matches "XMAS"

- : int = 18


In [54]:
read_wordsearch "day4_input.txt" |> count_matches "XMAS"

- : int = 2524


### Part 2

Oh good we have to search a different way.  Let's see...

In [55]:
let count_x_mas grid =
  let count_crossing_matches_at i j =
    try
      let ul = grid.(i-1).(j-1) in
      let ur = grid.(i-1).(j+1) in
      let dl = grid.(i+1).(j-1) in
      let dr = grid.(i+1).(j+1) in
      if Char.(grid.(i).(j) = 'A') &&
        (Char.(ul = 'M') && Char.(dr = 'S') ||
         Char.(ul = 'S') && Char.(dr = 'M')) &&
        (Char.(ur = 'M') && Char.(dl = 'S') ||
         Char.(ur = 'S') && Char.(dl = 'M')) then 1
      else 0
    with
      Invalid_argument _ -> 0 in
  let matches = ref 0 in
    let rows = Array.length grid in
    let cols = Array.length grid.(0) in
    for i = 0 to rows-1 do
      for j = 0 to cols-1 do
        let matches_here = count_crossing_matches_at i j in
          matches := !matches + matches_here
      done
    done; !matches

val count_x_mas : Core.Char.t Core.Array.t Core.Array.t -> int = <fun>


In [56]:
read_wordsearch "day4_input.txt" |>  count_x_mas

- : int = 1873


## <a name="day5">Day 5</a>

### Part 1

We're given a ~partial~ (seemingly total?) order as a bunch of pairs `x|y` which means x < y, and then a bunch of strings.  We have to check whether the strings are in order.  We'll just build an adjacency matrix in a hash table to store the ordering.

In [57]:
Hashtbl.of_alist_multi (module Int) [(47, 53); (97, 13); (97, 61); (97, 47)]

- : (Core.Int.t, int list) Core.Hashtbl.t = <abstr>


In [58]:
type day5_problem = {order: (Core.Int.t, int list) Core.Hashtbl.t; test_cases: int list list}

type day5_problem = {
  order : (Core.Int.t, int list) Core.Hashtbl.t;
  test_cases : int list list;
}


In [59]:
let read_day5_problem filename =
  let lines = In_channel.read_lines filename in
  let select char = filter ~f:(fun (line) -> String.contains line char) in
  let order =
    let parse line = String.split ~on:'|' line >>| int_of_string |> pair_of_list in
    lines |> select '|' >>| parse in
  let tests =
    let parse line = String.split ~on:',' line >>| int_of_string in
    lines |> select ',' >>| parse in
  {order = Hashtbl.of_alist_multi (module Int) order;
   test_cases = tests}

val read_day5_problem : Base.string -> day5_problem = <fun>


In [60]:
read_day5_problem "day5_example.txt"

- : day5_problem =
{order = <abstr>;
 test_cases =
  [[75; 47; 61; 53; 29]; [97; 61; 53; 29; 13]; [75; 29; 13];
   [75; 97; 47; 61; 53]; [61; 13; 29]; [97; 13; 75; 29; 47]]}


In [61]:
let order_lookup order x y =
  try
    Hashtbl.find_exn order x
    |> List.exists ~f:((=) y)
  with
  | Stdlib.Not_found | Not_found_s _ -> false

val order_lookup :
  ('a, Core_kernel.Int.t list) Core.Hashtbl.t ->
  'a Core.Hashtbl.key -> Core_kernel.Int.t -> bool = <fun>


In [62]:
let day5_compare order x y =
  if order_lookup order x y then -1
  else if order_lookup order y x then +1
  else 0

val day5_compare :
  (Core_kernel.Int.t, Core_kernel.Int.t list) Core.Hashtbl.t ->
  Core_kernel.Int.t Core.Hashtbl.key -> Core_kernel.Int.t -> int = <fun>


In [63]:
let in_order order xs =
  let len = length xs in
  let valid = ref true in
  (* It would be nicer to write this with a cartesian product that doesn't repeat pairs *)
  for i = 0 to len-2 do
    for j = i+1 to len-1 do
      if day5_compare order (nth_exn xs i) (nth_exn xs j) > 0 then
        valid := false
    done
  done; !valid

val in_order :
  (Core_kernel.Int.t, Core_kernel.Int.t list) Core.Hashtbl.t ->
  Core_kernel.Int.t Core.Hashtbl.key list -> bool = <fun>


In [64]:
let day5_part1 filename =
  let problem = read_day5_problem filename in
  let middle xs = nth_exn xs ((length xs) / 2) in
  let correctly_ordered = problem.test_cases
                        |> filter ~f:(in_order problem.order) in
  correctly_ordered >>| middle |> fold ~init:0 ~f:(+)

val day5_part1 : Base.string -> int = <fun>


In [65]:
day5_part1 "day5_example.txt"

- : int = 143


In [66]:
day5_part1 "day5_input.txt"

- : int = 6242


### Part 2

Now we have to sort stuff.

In [67]:
let day5_part2 filename =
  let problem = read_day5_problem filename in
  let middle xs = nth_exn xs ((length xs) / 2) in
  let incorrectly_ordered = problem.test_cases
                          |> filter ~f:(fun(xs) -> not (in_order problem.order xs)) in
  let compare x y = day5_compare problem.order x y in
  let sort xs = List.sort ~compare:compare xs in
  incorrectly_ordered >>| sort >>| middle |> fold ~init:0 ~f:(+)

val day5_part2 : Base.string -> int = <fun>


In [68]:
day5_part2 "day5_example.txt"

- : int = 123


In [69]:
day5_part2 "day5_input.txt"

- : int = 5169


## <a name="day6">Day 6</a>

### Part 1

Today we are plotting lines in a grid I guess.  Haven't we already read grids from files?  Maybe I will go factor out a helper function for that.

In [70]:
let read_map file_name = read_grid_from_file file_name

val read_map : Base.string -> char Core.Array.t Core.Array.t = <fun>


In [71]:
read_map "day6_example.txt"

- : char Core.Array.t Core.Array.t =
[|[|'.'; '.'; '.'; '.'; '#'; '.'; '.'; '.'; '.'; '.'|];
  [|'.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'|];
  [|'.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'|];
  [|'.'; '.'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'|];
  [|'.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '.'; '.'|];
  [|'.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'|];
  [|'.'; '#'; '.'; '.'; '^'; '.'; '.'; '.'; '.'; '.'|];
  [|'.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '.'|];
  [|'#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'|];
  [|'.'; '.'; '.'; '.'; '.'; '.'; '#'; '.'; '.'; '.'|]|]


Now we need to simulate a guard walking around.  Wow, ocaml's array library is really stilted for fp, or I am just not getting it.  I guess I will just keep it simple and use dumb imperative code.

In [72]:
let find_in_grid p grid =
  let pos = ref (-1, -1)
  and rows = Array.length grid
  and cols = Array.length grid.(0) in
  for i = 0 to rows-1 do
    for j = 0 to cols-1 do
      if p grid.(i).(j) then pos := (i, j)
    done
  done; !pos

val find_in_grid : ('a -> bool) -> 'a Core.Array.t Core.Array.t -> int * int =
  <fun>


In [73]:
read_map "day6_example.txt" |> find_in_grid (Char.(=) '^')

- : int * int = (6, 4)


I am going to have a whole terrible grid library by the end of the month...

In [74]:
let simulate_guard grid =
  let start = find_in_grid (Char.(=) '^') grid
  and turn dir = (snd dir, - fst dir)
  (* I guess Core kept the highly useful polymorphic stuff, but put it in "Poly" libraries... *)
  and count_distinct_positions path =
    Hash_set.Poly.of_list (path >>| fst) |> Hash_set.length in
  let rec move pos dir path =
    let forward = (fst pos + fst dir, snd pos + snd dir) in
    try
      match find path (Poly.(=) (forward, dir)) with
      | Some _ -> path
      | None ->
        match grid.(fst forward).(snd forward) with
        | '#' -> move pos (turn dir) ((pos, turn dir) :: path)
        | _ -> move forward dir ((forward, dir) :: path)
    with
      Invalid_argument _ -> path in
  let guard_path = move start (-1, 0) [(start, (-1, 0))] in
  count_distinct_positions guard_path

val simulate_guard : Core.Char.t Core.Array.t Core.Array.t -> int = <fun>


In [75]:
read_map "day6_example.txt" |> simulate_guard

- : int = 41


In [76]:
read_map "day6_input.txt" |> simulate_guard

- : int = 4663


### Part 2

Now we have to place obstacles to make the guard loop.  The grid is only 130 x 130 so we do not need to be especially clever - we'll just try all the possibilities.  But we should probably use a hash set directly to detect loops now.

In [77]:
let count_ways_to_loop_guard grid =
  let start = find_in_grid (Char.(=) '^') grid
  and turn dir = (snd dir, - fst dir)
  and rows = Array.length grid
  and cols = Array.length grid.(0) in
  let make_history pos = Hash_set.Poly.of_list [(pos, (-1, 0))] in
  let rec detect_loop obstacle pos dir history =
    let forward = (fst pos + fst dir, snd pos + snd dir) in
    try
      match Hash_set.mem history (forward, dir) with
      | true -> true
      | false ->
        match grid.(fst forward).(snd forward) with
        | '#' ->
          Hash_set.add history (pos, turn dir);
          detect_loop obstacle pos (turn dir) history
        | _ when Poly.(=) forward obstacle -> 
          Hash_set.add history (pos, turn dir);
          detect_loop obstacle pos (turn dir) history
        | _ ->
          Hash_set.add history (forward, dir);
          detect_loop obstacle forward dir history
    with
      Invalid_argument _ -> false in
  let ways = ref 0 in
  for i = 0 to rows-1 do
    for j = 0 to cols-1 do
      if (Poly.(<>) (i, j) start) && detect_loop (i,j) start (-1, 0) (make_history start) then incr ways
    done
  done; !ways

val count_ways_to_loop_guard : Core.Char.t Core.Array.t Core.Array.t -> int =
  <fun>


In [78]:
read_map "day6_example.txt" |> count_ways_to_loop_guard

- : int = 6


In [79]:
read_map "day6_input.txt" |> count_ways_to_loop_guard

- : int = 1530


## <a name="day7">Day 7</a>

### Part 1

Today we're reading ... possible equations I guess?

In [80]:
let read_equations filename =
  let parse line = String.split_on_chars ~on:[' '; ':'] line
                 |> filter ~f:(fun(s) -> not (String.is_empty s))
                 >>| int_of_string in
  In_channel.read_lines filename >>| parse

val read_equations : Base.string -> int list list = <fun>


In [81]:
read_equations "day7_example.txt"

- : int list list =
[[190; 10; 19]; [3267; 81; 40; 27]; [83; 17; 5]; [156; 15; 6];
 [7290; 6; 8; 6; 15]; [161011; 16; 10; 13]; [192; 17; 8; 14];
 [21037; 9; 7; 18; 13]; [292; 11; 6; 16; 20]]


We have to try inserting + and * operators and evaluating them left-to-right to make the equation work.  We'll use a bitmask to select which operator to insert at each position.  We need to use foldi to apply the operators, but I can't tell from the documentation which parameter is which in the function...

In [82]:
foldi [0; 0; 0; 0] ~init:0 ~f:(fun i result term -> result + i)

- : int = 6


In [83]:
let day7_part1 equations =
  let eval terms ops_mask =
    foldi (tl_exn terms)
      ~init:(hd_exn terms)
      ~f:(fun i result term ->
          match (Int.shift_left 1 i) land ops_mask with
          | 0 -> result + term
          | _ -> result * term) in
  let trial equation =
    let total = hd_exn equation in
    let terms = tl_exn equation in
    let num_operations = (length terms) - 1 in
    let result = ref 0 in
    for ops_mask=0 to (Int.shift_left 1 num_operations)-1 do
      if eval terms ops_mask = total then
        result := total
    done; !result in
  equations >>| trial |> fold ~init:0 ~f:(+)

val day7_part1 : Core_kernel.Int.t list list -> int = <fun>


In [84]:
read_equations "day7_example.txt" |> day7_part1

- : int = 3749


In [85]:
Int.shift_left 1 61 (* ok, I guess we probably have enough precision *)

- : Core.Int.t = 2305843009213693952


In [86]:
read_equations "day7_input.txt" |> day7_part1

- : int = 303876485655


### Part 2

Now we have to support a third type of operator that does digit concatenation.  This could make for some absurdly large numbers... the largest result in our data has 15 digits, so we'll just bail on anything larger.

In [87]:
let day7_part2 equations =
  let digit_cat a b =
    let sa = string_of_int a and sb = string_of_int b in
    if (String.length sa) + (String.length sb) > 15 then a + b
    else int_of_string (sa ^ sb) in
  let eval terms ops_mask =
    foldi (tl_exn terms)
      ~init:(hd_exn terms)
      ~f:(fun i result term ->
          match (ops_mask / (Int.pow 3 i)) mod 3 with
          | 2 -> digit_cat result term 
          | 1 -> result + term
          | _ -> result * term) in
  let trial equation =
    let total = hd_exn equation in
    let terms = tl_exn equation in
    let num_operations = (length terms) - 1 in
    let result = ref 0 in
    for ops_mask=0 to (Int.pow 3 num_operations)-1 do
      if eval terms ops_mask = total then
        result := total
    done; !result in
  equations >>| trial |> fold ~init:0 ~f:(+)

val day7_part2 : Core_kernel.Int.t list list -> int = <fun>


In [88]:
read_equations "day7_example.txt" |> day7_part2

- : int = 11387


In [89]:
read_equations "day7_input.txt" |> day7_part2

- : int = 146111650210682


## <a name="day8">Day 8</a>

### Part 1

Today we're trying to map out "antinodes", points which are colinear with two others.  The input is given as a grid, but it seems like we really just want some points.

In [90]:
type day8_antennas =
  {size: int;
   positions: (char, (int * int) list) Core.Hashtbl.Poly.t}

type day8_antennas = {
  size : int;
  positions : (char, (int * int) list) Core.Hashtbl.Poly.t;
}


In [91]:
let read_antennas filename =
  let grid = read_grid_from_file filename in
  let size = Array.length grid in
  let chars_and_points = ref [] in
    for i=0 to size-1 do
      for j=0 to size-1 do
        let here = grid.(i).(j) in
        if Char.(here <> '.') then
          chars_and_points := (here, (i, j)) :: !chars_and_points
      done
    done;
    {size=size;
     positions=Hashtbl.Poly.of_alist_multi !chars_and_points}

val read_antennas : Base.string -> day8_antennas = <fun>


In [92]:
let test_ant = read_antennas "day8_example.txt" in
  Hashtbl.data test_ant.positions

- : (int * int) list list =
[[(1, 8); (2, 5); (3, 7); (4, 4)]; [(5, 6); (8, 8); (9, 9)]]


We're going to want to look at pairs of points... this is the second time we've wanted a `cartesian_product` of distinct items, so let's just write that (I made a `pairs` helper up above).

In [93]:
let find_antinodes antennas =
  let size = antennas.size in
  let in_bounds (x1, y1) = x1 >= 0 && y1 >= 0 && x1 < size && y1 < size in
  let delta (x1, y1) (x2, y2) = (x1 - x2, y1 - y2) in
  let antinodes_for_points ((x1, y1), (x2, y2)) =
    let (dx, dy) = delta (x1, y1) (x2, y2) in
    [(x1 + dx, y1 + dy); (x2 - dx, y2 - dy)] in
  Hashtbl.data antennas.positions >>| pairs
  >>| concat_map ~f:antinodes_for_points |> concat
  |> filter ~f:in_bounds
  |> Hash_set.Poly.of_list |> Hash_set.length

val find_antinodes : day8_antennas -> int = <fun>


In [94]:
read_antennas "day8_example.txt" |> find_antinodes

- : int = 14


In [95]:
read_antennas "day8_input.txt" |> find_antinodes

- : int = 252


### Part 2

Now we have to find even more colinear points.  List comprehensions might be kind of nice... I guess I will use this crummy iota thing instead.

In [96]:
range (-5) 5

- : int list = [-5; -4; -3; -2; -1; 0; 1; 2; 3; 4]


In [97]:
let find_more_antinodes antennas =
  let size = antennas.size in
  let in_bounds (x1, y1) = x1 >= 0 && y1 >= 0 && x1 < size && y1 < size in
  let delta (x1, y1) (x2, y2) = (x1 - x2, y1 - y2) in
  let antinodes_for_points ((x1, y1), (x2, y2)) =
    let (dx, dy) = delta (x1, y1) (x2, y2) in
      (range (-size) size >>| fun d -> (x1 + d * dx, y1 + d * dy))
      |> filter ~f:in_bounds in
  Hashtbl.data antennas.positions >>| pairs
  >>| concat_map ~f:antinodes_for_points |> concat
  |> Hash_set.Poly.of_list |> Hash_set.length

val find_more_antinodes : day8_antennas -> int = <fun>


In [98]:
read_antennas "day8_example.txt" |> find_more_antinodes

- : int = 34


In [99]:
read_antennas "day8_input.txt" |> find_more_antinodes

- : int = 839


## <a name="day9">Day 9</a>

### Part 1

We're defragging a hard drive today.  Man, it has been a few years.

In [100]:
type hd_block =
  | Day9File of int
  | Day9FreeSpace

type hd_block = Day9File of int | Day9FreeSpace


In [101]:
let read_hd_map filename =
  let digit c = int_of_char c - int_of_char '0' in
  let unpack index size =
    List.init size ~f:(fun _ ->
      if index mod 2 = 0 then Day9File (index / 2)
      else Day9FreeSpace) in
  In_channel.read_all filename
  |> String.to_list
  >>| digit
  |> concat_mapi ~f:unpack
  |> Array.of_list

val read_hd_map : Base.string -> hd_block Core.Array.t = <fun>


In [102]:
read_hd_map "day9_example.txt"

- : hd_block Core.Array.t =
[|Day9File 0; Day9File 0; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace;
  Day9File 1; Day9File 1; Day9File 1; Day9FreeSpace; Day9FreeSpace;
  Day9FreeSpace; Day9File 2; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace;
  Day9File 3; Day9File 3; Day9File 3; Day9FreeSpace; Day9File 4; Day9File 4;
  Day9FreeSpace; Day9File 5; Day9File 5; Day9File 5; Day9File 5;
  Day9FreeSpace; Day9File 6; Day9File 6; Day9File 6; Day9File 6;
  Day9FreeSpace; Day9File 7; Day9File 7; Day9File 7; Day9FreeSpace;
  Day9File 8; Day9File 8; Day9File 8; Day9File 8; Day9File 9; Day9File 9|]


In [103]:
let defrag hd =
  let len = Array.length hd in
  let i = ref 0 in
  let j = ref (len-1) in
  while !i < !j do
    match hd.(!i), hd.(!j) with
    | Day9File _, _ -> incr i
    | _, Day9FreeSpace -> decr j
    | Day9FreeSpace, Day9File _ -> begin
        hd.(!i) <- hd.(!j);
        hd.(!j) <- Day9FreeSpace;
        incr i; decr j
      end
  done; hd

val defrag : hd_block Core.Array.t -> hd_block Core.Array.t = <fun>


In [104]:
read_hd_map "day9_example.txt" |> defrag

- : hd_block Core.Array.t =
[|Day9File 0; Day9File 0; Day9File 9; Day9File 9; Day9File 8; Day9File 1;
  Day9File 1; Day9File 1; Day9File 8; Day9File 8; Day9File 8; Day9File 2;
  Day9File 7; Day9File 7; Day9File 7; Day9File 3; Day9File 3; Day9File 3;
  Day9File 6; Day9File 4; Day9File 4; Day9File 6; Day9File 5; Day9File 5;
  Day9File 5; Day9File 5; Day9File 6; Day9File 6; Day9FreeSpace;
  Day9FreeSpace; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace;
  Day9FreeSpace; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace;
  Day9FreeSpace; Day9FreeSpace; Day9FreeSpace|]


In [105]:
let checksum hd_map =
  Array.foldi hd_map ~init:0 ~f:(fun i total contents ->
    match contents with
    | Day9File num -> total + i * num
    | _ -> total)

val checksum : hd_block Core.Array.t -> int = <fun>


In [106]:
read_hd_map "day9_example.txt" |> defrag |> checksum

- : int = 1928


In [107]:
read_hd_map "day9_input.txt" |> defrag |> checksum

- : int = 6360094256423


### Part 2

Now we need to defrag not quite as badly I guess?  We need to keep track of a few auxiliary data structures for this part, and it is simpler to build them when reading the file, so we'll start over.

In [108]:
type hd = {
  map: hd_block Core.Array.t;
  directory: (int, int * int) Core.Hashtbl.Poly.t;
  free_space_of_size: (int, int list) Core.Hashtbl.Poly.t;
  last_file: int
}

type hd = {
  map : hd_block Core.Array.t;
  directory : (int, int * int) Core.Hashtbl.Poly.t;
  free_space_of_size : (int, int list) Core.Hashtbl.Poly.t;
  last_file : int;
}


In [109]:
let read_hd filename =
  let digit c = int_of_char c - int_of_char '0' in
  let unpack_map index size =
    List.init size ~f:(fun _ ->
      if index mod 2 = 0 then Day9File (index / 2)
      else Day9FreeSpace) in
  let unpack_directory index base size =
    if index mod 2 = 0 then (base + size, Some (index / 2, (base, size)))
    else (base + size, None) in
  let unpack_free_space index base size =
    if index mod 2 = 1 && size > 0 then (base + size, Some (size, base))
    else (base + size, None) in
  let digits = In_channel.read_all filename
             |> String.to_list
             >>| digit in
  let map = digits |> concat_mapi ~f:unpack_map |> Array.of_list in
  let directory = digits |> folding_mapi ~init:0 ~f:unpack_directory |> filter_opt
                |> Hashtbl.Poly.of_alist_exn in
  let free_space = digits |> folding_mapi ~init:0 ~f:unpack_free_space |> filter_opt
                 |> Hashtbl.Poly.of_alist_multi
                 |> Hashtbl.map ~f:(fun xs -> sort ~compare:Int.compare xs) in
  {map=map;
   directory=directory;
   free_space_of_size=free_space;
   last_file=((length digits) - 1) / 2}

val read_hd : Base.string -> hd = <fun>


In [110]:
Hashtbl.data (read_hd "day9_example.txt").free_space_of_size

- : int list list = [[2; 8; 12]; [18; 21; 26; 31; 35]]


This took me way too long because I didn't understand that files could still only move to the left of their original position.  I guess we wouldn't really accomplish defragmenting otherwise...

In [111]:
let defrag2 hd =
  let first_free_of_size before_pos size =
    match Hashtbl.find hd.free_space_of_size size with
    | Some (pos::_) when pos < before_pos -> Some (size, pos)
    | _ -> None in
  let add_free_space size base =
    Hashtbl.add_multi hd.free_space_of_size size base;
    let blocks = Hashtbl.find_exn hd.free_space_of_size size in
    let sorted_blocks = sort blocks ~compare:Int.compare in
    Hashtbl.set hd.free_space_of_size size sorted_blocks in
  let move_file id =
    let (base, size) = Hashtbl.find_exn hd.directory id in
    let valid_sizes = range size 10 in
    let choices = valid_sizes >>| first_free_of_size base |> filter_opt in
    let dest = choices |> min_elt ~compare:(fun (_, x) (_, y) -> Int.compare x y) in
      match dest with
      | Some (dest_size, dest_pos) -> begin
          for i = 0 to size-1 do
            hd.map.(base + i) <- Day9FreeSpace;
            match hd.map.(dest_pos + i) with
            | Day9FreeSpace -> hd.map.(dest_pos + i) <- Day9File id
            | _ -> failwith "overwriting file"
          done;
          Hashtbl.set hd.directory id (dest_pos, size);
          (* the block we just took must be the first *)
          Hashtbl.remove_multi hd.free_space_of_size dest_size;
          (* we don't need this because of the constraint about only moving files to the left *)
          (*add_free_space size base;*)
          if dest_size > size then add_free_space (dest_size - size) (dest_pos + size);
        end
      | None -> () in
  for id=hd.last_file downto 0 do
    move_file id
  done; hd

val defrag2 : hd -> hd = <fun>


In [112]:
read_hd "day9_example.txt" |> defrag2

- : hd =
{map =
  [|Day9File 0; Day9File 0; Day9File 9; Day9File 9; Day9File 2; Day9File 1;
    Day9File 1; Day9File 1; Day9File 7; Day9File 7; Day9File 7;
    Day9FreeSpace; Day9File 4; Day9File 4; Day9FreeSpace; Day9File 3;
    Day9File 3; Day9File 3; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace;
    Day9FreeSpace; Day9File 5; Day9File 5; Day9File 5; Day9File 5;
    Day9FreeSpace; Day9File 6; Day9File 6; Day9File 6; Day9File 6;
    Day9FreeSpace; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace;
    Day9FreeSpace; Day9File 8; Day9File 8; Day9File 8; Day9File 8;
    Day9FreeSpace; Day9FreeSpace|];
 directory = <abstr>; free_space_of_size = <abstr>; last_file = 9}


In [113]:
(read_hd "day9_example.txt" |> defrag2).map |> checksum

- : int = 2858


In [114]:
(read_hd "day9_input.txt" |> defrag2).map |> checksum

- : int = 6379677752410


## <a name="day10">Day 10</a>

### Part 1

Hooray, we are searching in a grid.  I love searching in grids.  Yup.

In [115]:
let read_topo_map filename =
  let map = read_grid_from_file filename in
  let digit_of_char c = int_of_char c - int_of_char '0' in
  map |> Array.map ~f:(fun row -> Array.map row digit_of_char)

val read_topo_map : Base.string -> int Core.Array.t Core.Array.t = <fun>


In [116]:
read_topo_map "day10_example.txt"

- : int Core.Array.t Core.Array.t =
[|[|8; 9; 0; 1; 0; 1; 2; 3|]; [|7; 8; 1; 2; 1; 8; 7; 4|];
  [|8; 7; 4; 3; 0; 9; 6; 5|]; [|9; 6; 5; 4; 9; 8; 7; 4|];
  [|4; 5; 6; 7; 8; 9; 0; 3|]; [|3; 2; 0; 1; 9; 0; 1; 2|];
  [|0; 1; 3; 2; 9; 8; 0; 1|]; [|1; 0; 4; 5; 6; 7; 3; 2|]|]


In [117]:
let grid_find_all grid ~x =
  let rows = Array.length grid in
  let cols = Array.length grid.(0) in
  let matches = ref [] in
  for i=0 to rows-1 do
    for j=0 to cols-1 do
      if grid.(i).(j) = x then
        matches := (i, j) :: !matches
      done
    done; !matches

val grid_find_all :
  Core_kernel.Int.t Core.Array.t Core.Array.t ->
  x:Core_kernel.Int.t -> (int * int) list = <fun>


In [118]:
read_topo_map "day10_example.txt" |> grid_find_all ~x:0

- : (int * int) list =
[(7, 1); (6, 6); (6, 0); (5, 5); (5, 2); (4, 6); (2, 4); (0, 4); (0, 2)]


In [119]:
let score_trails map =
  let size = Array.length map in
  let trailheads = grid_find_all map 0 in
  let neighbors (i,j) =
    (if i > 0 && map.(i-1).(j) = 1 + map.(i).(j) then [(i-1, j)] else []) @
    (if j > 0 && map.(i).(j-1) = 1 + map.(i).(j) then [(i, j-1)] else []) @
    (if i < size-1 && map.(i+1).(j) = 1 + map.(i).(j) then [(i+1, j)] else []) @
    (if j < size-1 && map.(i).(j+1) = 1 + map.(i).(j) then [(i, j+1)] else []) in
  let rec search horizon =
    match horizon with
    | [] -> []
    | [(i,j)] ->
        if map.(i).(j) = 9 then [(i, j)]
        else search (neighbors (i, j))
    | (i,j)::rest ->
        if map.(i).(j) = 9 then (i,j) :: search rest
        else search (neighbors (i,j) @ rest) in
  let compare_tuples (a1,b1) (a2,b2) =
    if a1 = a2 then Stdlib.compare b1 b2
    else Stdlib.compare a1 a2 in
  let count_distinct_reachable_9s p =
    search [p] |> dedup_and_sort ~compare:compare_tuples |> length in
  (* part 2 *)
  let count_trails p =
    search [p] |> length in
  let sum = fold ~init:0 ~f:(+) in
  (trailheads >>| count_distinct_reachable_9s |> sum,
   trailheads >>| count_trails |> sum)

val score_trails : Core_kernel.Int.t Core.Array.t Core.Array.t -> int * int =
  <fun>


In [120]:
read_topo_map "day10_example.txt" |> score_trails

- : int * int = (36, 81)


In [121]:
read_topo_map "day10_input.txt" |> score_trails

- : int * int = (593, 1192)


### Part 2

I happened to write my search to find all paths anyway so just added an extra output to the tuple for part 1.

## <a name="day11">Day 11</a>

### Part 1

Today is a good day because it is not a grid day.  Instead, today, we've got... stones?  Big stones, apparently.  Is there event a bigint library?  Yes... ok, it's so old that it doesn't work with the current package manager.

In [122]:
#load "nums.cma" ;;

In [123]:
let read_stones filename =
  let trim s =
    let len = String.length s in
    if Char.(s.[len - 1] = '\n') then String.sub s 0 (len - 1)
    else s in
  In_channel.read_all filename
  |> trim
  |> String.split ~on:' '

val read_stones : Base.string -> Base.string list = <fun>


In [124]:
read_stones "day11_example.txt"

- : Base.string list = ["125"; "17"]


In [125]:
let split_digits s =
  let len = String.length s in
  if len mod 2 = 0 then
    Some [Big_int.(string_of_big_int @@ big_int_of_string @@ String.sub s 0 (len/2));
          Big_int.(string_of_big_int @@ big_int_of_string @@ String.sub s (len/2) (len/2))]
  else
    None

val split_digits : string -> string list option = <fun>


In [126]:
let step_stones s =
  if String.(s = "0") then ["1"]
  else match split_digits s with
  | Some split -> split
  | None -> [Big_int.(mult_int_big_int 2024 (big_int_of_string s) |> string_of_big_int)]

val step_stones : Core.String.t -> string list = <fun>


In [127]:
let blink stones = stones |> concat_map ~f:step_stones

val blink : Core.String.t list -> string list = <fun>


In [128]:
let rec iterate n fn acc =
  if n = 0 then acc
  else iterate (n-1) fn (fn acc)

val iterate : Core_kernel.Int.t -> ('a -> 'a) -> 'a -> 'a = <fun>


In [129]:
read_stones "day11_example.txt" |> iterate 6 blink

- : Base.string list =
["2097446912"; "14168"; "4048"; "2"; "0"; "2"; "4"; "40"; "48"; "2024"; "40";
 "48"; "80"; "96"; "2"; "8"; "6"; "7"; "6"; "0"; "3"; "2"]


In [130]:
read_stones "day11_example.txt" |> iterate 25 blink |> length

- : int = 55312


In [131]:
read_stones "day11_input.txt" |> iterate 25 blink |> length

- : int = 220722


### Part 2

Now we have to iterate 75 times.  Let's see if we can just do this.

In [132]:
(* read_stones "day11_input.txt" |> iterate 75 blink |> length *)

Ok, I guess we have to think harder.  How many _distinct_ things do we have after N iterations?

In [133]:
read_stones "day11_input.txt"
|> iterate 25 blink
|> Hash_set.Poly.of_list
|> Hash_set.length

- : int = 380


Great, so let's keep a census of types of stones instead of repeating all that work.  Except Big_ints aren't hashable, because of course... so let's go back and use strings for everything, SIGH.

In [134]:
let update_stone_count ~census ~count kind =
  Hashtbl.update census kind ~f:(fun n -> match n with
    | Some n -> n + count
    | None -> count)

val update_stone_count :
  census:('a, int) Core.Hashtbl.t -> count:int -> 'a Core.Hashtbl.key -> unit =
  <fun>


In [135]:
let make_initial_census stones =
  let census = Hashtbl.create (module String) in
  let _ = stones >>| update_stone_count ~census:census ~count:1 in
  census

val make_initial_census :
  Core.String.t Core.Hashtbl.key list -> (Core.String.t, int) Core.Hashtbl.t =
  <fun>


In [136]:
let blink_smarter census =
  let census' = Hashtbl.create (module String) in
  let _ = Hashtbl.iteri census ~f:(fun ~key ~data ->
    iter (step_stones key) (update_stone_count ~census:census' ~count:data)
  ) in
  census'

val blink_smarter :
  (Core.String.t, int) Core.Hashtbl.t -> (Core.String.t, int) Core.Hashtbl.t =
  <fun>


In [137]:
read_stones "day11_input.txt"
|> make_initial_census
|> iterate 25 blink_smarter
|> Hashtbl.fold ~init:0 ~f:(fun ~key ~data acc -> acc + data)

- : int = 220722


In [138]:
read_stones "day11_input.txt"
|> make_initial_census
|> iterate 75 blink_smarter
|> Hashtbl.fold ~init:0 ~f:(fun ~key ~data acc -> acc + data)

- : int = 261952051690787


## <a name="day12">Day 12</a>

### Part 1

Sigh, here we go again with grids.  Today we are flood filling regions I guess.

In [139]:
read_grid_from_file "day12_example.txt"

- : char Core.Array.t Core.Array.t =
[|[|'R'; 'R'; 'R'; 'R'; 'I'; 'I'; 'C'; 'C'; 'F'; 'F'|];
  [|'R'; 'R'; 'R'; 'R'; 'I'; 'I'; 'C'; 'C'; 'C'; 'F'|];
  [|'V'; 'V'; 'R'; 'R'; 'R'; 'C'; 'C'; 'F'; 'F'; 'F'|];
  [|'V'; 'V'; 'R'; 'C'; 'C'; 'C'; 'J'; 'F'; 'F'; 'F'|];
  [|'V'; 'V'; 'V'; 'V'; 'C'; 'J'; 'J'; 'C'; 'F'; 'E'|];
  [|'V'; 'V'; 'I'; 'V'; 'C'; 'C'; 'J'; 'J'; 'E'; 'E'|];
  [|'V'; 'V'; 'I'; 'I'; 'I'; 'C'; 'J'; 'J'; 'E'; 'E'|];
  [|'M'; 'I'; 'I'; 'I'; 'I'; 'I'; 'J'; 'J'; 'E'; 'E'|];
  [|'M'; 'I'; 'I'; 'I'; 'S'; 'I'; 'J'; 'E'; 'E'; 'E'|];
  [|'M'; 'M'; 'M'; 'I'; 'S'; 'S'; 'J'; 'E'; 'E'; 'E'|]|]


Ok, apparently we only have core v0.14.1, because that's the latest version that is in opam?  So I can't copy a matrix.

In [140]:
Array.copy_matrix

error: compile_error

Fine, we'll do it live.

In [141]:
let copy_matrix matrix =
  let rows = Array.length matrix in
  let cols = Array.length matrix.(0) in
  let copy = Array.make_matrix rows cols matrix.(0).(0) in
  for i=0 to rows-1 do
    for j=0 to cols-1 do
      copy.(i).(j) <- matrix.(i).(j)
    done
  done; copy

val copy_matrix :
  'a Core.Array.t Core.Array.t -> 'a Core.Array.t Core.Array.t = <fun>


In [142]:
let find_plant_groups grid =
  (*let plots = Array.copy_matrix grid in*)
  let plots = copy_matrix grid in
  let size = Array.length plots in
  let rec fill plant i j =
    if i < 0 || i > size-1 || j < 0 || j > size-1 then []
    else if Char.(plant = plots.(i).(j)) then begin
      plots.(i).(j) <- '.';
      (i,j) :: (
        fill plant (i-1) j @
        fill plant (i+1) j @
        fill plant i (j-1) @
        fill plant i (j+1))
    end else [] in
  let regions = ref [] in
  for i=0 to size-1 do
    for j=0 to size-1 do
      let plant = plots.(i).(j) in
      if Char.(plant <> '.') then
        regions := (fill plant i j) :: !regions
    done
  done; !regions

val find_plant_groups :
  Core.Char.t Core.Array.t Core.Array.t ->
  (Core_kernel.Int.t * Core_kernel.Int.t) Base.List.t list = <fun>


In [143]:
read_grid_from_file "day12_example.txt" |> find_plant_groups >>| length

- : int list = [3; 5; 14; 13; 1; 11; 13; 10; 14; 4; 12]


In [144]:
let sum = fold ~init:0 ~f:(+)

val sum : int list -> int = <fun>


In [145]:
let fence_cost grid =
  let size = Array.length grid in
  let groups = find_plant_groups grid in
  let fenced_sides (i,j) =
    (if i = 0 || Char.(grid.(i-1).(j) <> grid.(i).(j)) then 1 else 0) +
    (if i = size-1 || Char.(grid.(i+1).(j) <> grid.(i).(j)) then 1 else 0) +
    (if j = 0 || Char.(grid.(i).(j-1) <> grid.(i).(j)) then 1 else 0) +
    (if j = size-1 || Char.(grid.(i).(j+1) <> grid.(i).(j)) then 1 else 0) in
  let perimeter group = group >>| fenced_sides |> sum in
  let area group = length group in
  let cost group = (perimeter group) * (area group) in
  groups >>| cost |> sum

val fence_cost : Core.Char.t Core.Array.t Core.Array.t -> int = <fun>


In [146]:
read_grid_from_file "day12_example.txt" |> fence_cost

- : int = 1930


In [147]:
read_grid_from_file "day12_input.txt" |> fence_cost

- : int = 1304764


### Part 2

Now instead we want to count the edges of each group.  For example, two vertically adjacent plants can share the same west fence, so if a plant needs a west fence but its north neighbor already has a west fence, we'll skip counting it.

In [148]:
let fence_cost2 grid =
  let size = Array.length grid in
  let groups = find_plant_groups grid in
  let needs_fence (i, j) (di, dj) =
    let (i', j') = (i + di, j + dj) in
    if i' < 0 || i' > size - 1 || j' < 0 || j' > size - 1 then true
    else Char.(grid.(i).(j) <> grid.(i').(j')) in
  let cant_share_horizontal_fence (i, j) dir =
    i = 0 ||
    Char.(grid.(i).(j) <> grid.(i - 1).(j)) ||
    not (needs_fence (i - 1, j) dir) in
  let cant_share_vertical_fence (i, j) dir =
    j = 0 ||
    Char.(grid.(i).(j) <> grid.(i).(j - 1)) ||
    not (needs_fence (i, j - 1) dir) in
  let fenced_sides (i, j) =
    (if needs_fence (i, j) (0, -1) &&
        cant_share_horizontal_fence (i, j) (0, -1) then 1
     else 0) +
    (if needs_fence (i, j) (0, 1) &&
        cant_share_horizontal_fence (i, j) (0, 1) then 1
     else 0) +
    (if needs_fence (i, j) (-1, 0) &&
        cant_share_vertical_fence (i, j) (-1, 0) then 1
     else 0) +
    (if needs_fence (i, j) (1, 0) &&
        cant_share_vertical_fence (i, j) (1, 0) then 1
     else 0) in
  let perimeter group = group >>| fenced_sides |> sum in
  let area group = length group in
  let cost group = (perimeter group) * (area group) in
  groups >>| cost |> sum

val fence_cost2 : Core.Char.t Core.Array.t Core.Array.t -> int = <fun>


In [149]:
read_grid_from_file "day12_example1.txt"

- : char Core.Array.t Core.Array.t =
[|[|'A'; 'A'; 'A'; 'A'|]; [|'B'; 'B'; 'C'; 'D'|]; [|'B'; 'B'; 'C'; 'C'|];
  [|'E'; 'E'; 'E'; 'C'|]|]


In [150]:
read_grid_from_file "day12_example1.txt" |> fence_cost2

- : int = 80


In [151]:
read_grid_from_file "day12_example2.txt" |> fence_cost2

- : int = 236


In [152]:
read_grid_from_file "day12_example3.txt" |> fence_cost2

- : int = 368


In [153]:
read_grid_from_file "day12_example.txt" |> fence_cost2

- : int = 1206


In [154]:
read_grid_from_file "day12_input.txt" |> fence_cost2

- : int = 811148


## <a name="day13">Day 13</a>

### Part 1

Today looks like it's going to be an integer programming problem, but for the first part we just have to solve some simultaneous equations... for example, $94 a + 22 b = 8400$ and $34 a + 67 b = 5400$.  The input is in a really annoying to parse format.

In [155]:
(*read records like
Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400
*)
let read_claw_problems filename =
  let problems =
    In_channel.read_lines filename
    |> filter ~f:(fun s -> String.length s > 0)
    |> chunks_of ~length:3 in
  let extract_xy re line =
    let m = Re2.first_match_exn re line in 
    let x = Re2.Match.get_exn m ~sub:(`Index 1) in
    let y = Re2.Match.get_exn m ~sub:(`Index 2) in
      (int_of_string x, int_of_string y) in
  let parse_button_line line =
    extract_xy (Re2.create_exn {|Button .: X\+(\d+), Y\+(\d+)|}) line in
  let parse_prize_line line =
    extract_xy (Re2.create_exn {|Prize: X=(\d+), Y=(\d+)|}) line in
  let parse_chunk chunk =
    let (ax, ay) = parse_button_line (List.nth_exn chunk 0) in
    let (bx, by) = parse_button_line (List.nth_exn chunk 1) in
    let (px, py) = parse_prize_line (List.nth_exn chunk 2) in
    let xs = [|[|ax; bx|];
               [|ay; by|]|] in
    let ys = [|[|px|];
               [|py|]|] in
      (xs, ys) in
  problems >>| parse_chunk

val read_claw_problems :
  Base.string -> (int array array * int array array) list = <fun>


In [156]:
read_claw_problems "day13_example.txt"

- : (int array array * int array array) list =
[([|[|94; 22|]; [|34; 67|]|], [|[|8400|]; [|5400|]|]);
 ([|[|26; 67|]; [|66; 21|]|], [|[|12748|]; [|12176|]|]);
 ([|[|17; 84|]; [|86; 37|]|], [|[|7870|]; [|6450|]|]);
 ([|[|69; 27|]; [|23; 71|]|], [|[|18641|]; [|10279|]|])]


In [157]:
let solve22 (a, y) =
  let det m = m.(0).(0) * m.(1).(1) - m.(0).(1) * m.(1).(0) in
  let d = det a in
  let dx = det [|[|y.(0).(0); a.(0).(1)|];
                 [|y.(1).(0); a.(1).(1)|]|] in
  let dy = det [|[|a.(0).(0); y.(0).(0)|];
                 [|a.(1).(0); y.(1).(0)|]|] in
  if d = 0 then None
  else if dx mod d <> 0 || dy mod d <> 0 then None
  else Some (dx / d, dy / d)

val solve22 :
  int Core.Array.t Core.Array.t * int Core.Array.t Core.Array.t ->
  (int * int) option = <fun>


In [158]:
let day13_cost problems =
  let cost (a, b) = 3 * a + b in
  problems >>| solve22 |> filter_opt >>| cost |> fold ~init:0 ~f:(+)

val day13_cost :
  (int Core.Array.t Core.Array.t * int Core.Array.t Core.Array.t) list -> int =
  <fun>


In [159]:
read_claw_problems "day13_example.txt" |> day13_cost

- : int = 480


In [160]:
read_claw_problems "day13_input.txt" |> day13_cost

- : int = 31623


### Part 2

Now we need to add 10000000000000 to some constants and ... do the same thing as before?  What a weird problem.

In [161]:
let add_10000000000000 (a, y) =
  (a, [|[|y.(0).(0) + 10000000000000|];
        [|y.(1).(0) + 10000000000000|]|])

val add_10000000000000 :
  'a * int Core.Array.t Core.Array.t -> 'a * int array array = <fun>


In [162]:
read_claw_problems "day13_input.txt" >>| add_10000000000000 |> day13_cost

- : int = 93209116744825


## <a name="day14">Day 14</a>

### Part 1

Robots.  Hookay.

In [163]:
type robot = {
  position: int * int;
  velocity: int * int
}

type robot = { position : int * int; velocity : int * int; }


In [164]:
let read_robots filename =
  let parse line =
    let re = Re2.create_exn {|p=(\d+),(\d+) v=(-?\d+),(-?\d+)|} in
    let m = Re2.first_match_exn re line in 
    let px = int_of_string @@ Re2.Match.get_exn m ~sub:(`Index 1) in
    let py = int_of_string @@ Re2.Match.get_exn m ~sub:(`Index 2) in
    let vx = int_of_string @@ Re2.Match.get_exn m ~sub:(`Index 3) in
    let vy = int_of_string @@ Re2.Match.get_exn m ~sub:(`Index 4) in
      {position=(px, py); velocity=(vx, vy)} in
  In_channel.read_lines filename >>| parse

val read_robots : Base.string -> robot list = <fun>


In [165]:
read_robots "day14_example.txt"

- : robot list =
[{position = (0, 4); velocity = (3, -3)};
 {position = (6, 3); velocity = (-1, -3)};
 {position = (10, 3); velocity = (-1, 2)};
 {position = (2, 0); velocity = (2, -1)};
 {position = (0, 0); velocity = (1, 3)};
 {position = (3, 0); velocity = (-2, -2)};
 {position = (7, 6); velocity = (-1, -3)};
 {position = (3, 0); velocity = (-1, -2)};
 {position = (9, 3); velocity = (2, 3)};
 {position = (7, 3); velocity = (-1, 2)};
 {position = (2, 4); velocity = (2, -3)};
 {position = (9, 5); velocity = (-3, -3)}]


In [166]:
let step_robot robot width height =
  let {position; velocity} = robot in
  let (px, py), (vx, vy) = position, velocity in
  let rec wrap v m =
    if v < 0 then wrap (v + m) m
    else v mod m in
  {position=(wrap (px + vx) width, wrap (py + vy) height);
   velocity=velocity}

val step_robot : robot -> int -> int -> robot = <fun>


In [167]:
iterate 5 (fun r -> step_robot r 11 7) {position = (2, 4); velocity = (2, -3)}

- : robot = {position = (1, 3); velocity = (2, -3)}


In [168]:
let simulate_robots width height steps robots =
  let simulate r = step_robot r width height in
  robots >>| iterate steps simulate

val simulate_robots :
  int -> int -> Core_kernel.Int.t -> robot list -> robot list = <fun>


In [169]:
let count_quadrants width height robots =
  let quadrant {position=(px, py)} = (px < width / 2, py < height / 2) in
  let not_in_middle {position=(px, py)} = px <> width / 2 && py <> height / 2 in
  let quads = robots
            |> filter ~f:not_in_middle
            >>| quadrant
            |> sort ~compare:Stdlib.compare
            |> group ~break:Poly.(<>)
            >>| length in
  if length quads < 4 then 0
  else fold quads ~init:1 ~f:( * )

val count_quadrants : int -> int -> robot list -> int = <fun>


In [170]:
read_robots "day14_example.txt" |> simulate_robots 11 7 100 |> count_quadrants 11 7

- : int = 12


In [171]:
read_robots "day14_input.txt" |> simulate_robots 101 103 100 |> count_quadrants 101 103

- : int = 211773366


### Part 2

Ok this is extremely cool... apparently we need to plot these points and see when they look like a Christmas tree.

In [172]:
let plot out width height robots =
  for y=0 to height-1 do
    for x=0 to width-1 do
      let ch = if exists robots ~f:(fun {position=(px,py)} -> px = x && py = y) then '*' else '.' in
        Out_channel.output_char out ch
    done; Out_channel.output_char out '\n'
  done

val plot : Core.Out_channel.t -> int -> int -> robot list -> unit = <fun>


In [173]:
let simulate_and_plot filename width height plot_from steps robots =
  let out = Out_channel.create filename in
  let simulate r = step_robot r width height in
  let robots = ref robots in
  for i=0 to steps do
    if i > plot_from then begin
      Out_channel.output_string out ("===== " ^ (string_of_int i) ^ " =====\n");
      plot out width height !robots
    end;
    robots := !robots >>| simulate
  done;
  Out_channel.close out

val simulate_and_plot :
  Base.string ->
  int -> int -> Core_kernel.Int.t -> int -> robot list -> Base.unit = <fun>


Whatever I'm looking for doesn't seem to happen in the first 1200 or so generations of the example input, which is sort of annoying.  This notebook environment isn't really helping, since I can't easily page through... let's try writing it out to a file.

In [174]:
(*read_robots "day14_example.txt" |> simulate_and_plot "day14_example_plot.txt" 11 7 (-1) 10000*)

Oh, that's embarrassing.  The example loops after 77 steps, and never _really_ seems to look like a tree.  Ok, I don't have a ton of hope here, but let's glance at the full data.

In [175]:
(*read_robots "day14_input.txt" |> simulate_and_plot "day14_example_plot.txt" 101 103 (-1) 10000*)

Things look... clumpier sometimes?  Like at step 31, there's a horizontal clump.  At 72, there's a vertical clump.  Let's look at when clumps suddenly appear.

| H Clump | V Clump |
| ------- | ------- |
|      31 |      72 |
|     134 |     173 |
|     237 |     274 |

It looks like $step_h = 31 + 103n$ and $step_v = 72 + 101n$.  Let's predict and check a few more clumpy steps...

In [176]:
range 0 10 >>| fun n -> 31 + 103 * n

- : int list = [31; 134; 237; 340; 443; 546; 649; 752; 855; 958]


In [177]:
range 0 10 >>| fun n -> 72 + 101 * n

- : int list = [72; 173; 274; 375; 476; 577; 678; 779; 880; 981]


The pattern seems to hold.  On what step should the h clumps and v clumps first coincide?

In [178]:
let overlap =
  let h_clumps n = 31 + 103 * n in
  let v_clumps n = 72 + 101 * n in
  let hs = range 0 100 >>| h_clumps in
  let vs = range 0 100 >>| v_clumps in
  find hs ~f:(fun k -> exists vs ~f:((=) k))

val overlap : Core_kernel.Int.t option = Some 7344


In [179]:
read_robots "day14_input.txt" |> simulate_and_plot "day14_example_plot.txt" 101 103 7343 7345

- : Base.unit = ()


Ok yeah there's pretty definitively a Christmas tree hidden in here.  What a cool puzzle, best part 2 ever.

## <a name="day15">Day 15</a>

### Part 1

Today we have to play Sokoban.  Ok, this could be fun.  But first... we must read the input.

In [180]:
type sokoban = {
  grid: char Core.Array.t Core.Array.t;
  robot: int * int;
  moves: char list
}

type sokoban = {
  grid : char Core.Array.t Core.Array.t;
  robot : int * int;
  moves : char list;
}


In [181]:
let read_sokoban filename =
  let lines = In_channel.read_lines filename in
  let size = String.length (hd_exn lines) in
  let robot = ref (0, 0) in
  let grid = Array.make_matrix size size '.' in
  let () =
    for i = 0 to size-1 do
      for j = 0 to size-1 do
        let here = String.get (nth_exn lines i) j in
          if Char.(here = '@') then begin
            robot := (i, j);
            grid.(i).(j) <- '.'
          end else
            grid.(i).(j) <- here
      done
    done in
  let moves = List.drop lines (size + 1)
            |> fold ~init:"" ~f:String.(^)
            |> String.to_list in
  {grid = grid; robot = !robot; moves = moves}

val read_sokoban : Base.string -> sokoban = <fun>


In [182]:
read_sokoban "day15_example.txt"

- : sokoban =
{grid =
  [|[|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|];
    [|'#'; '.'; '.'; 'O'; '.'; '.'; 'O'; '.'; 'O'; '#'|];
    [|'#'; '.'; '.'; '.'; '.'; '.'; '.'; 'O'; '.'; '#'|];
    [|'#'; '.'; 'O'; 'O'; '.'; '.'; 'O'; '.'; 'O'; '#'|];
    [|'#'; '.'; '.'; 'O'; '.'; '.'; '.'; 'O'; '.'; '#'|];
    [|'#'; 'O'; '#'; '.'; '.'; 'O'; '.'; '.'; '.'; '#'|];
    [|'#'; 'O'; '.'; '.'; 'O'; '.'; '.'; 'O'; '.'; '#'|];
    [|'#'; '.'; 'O'; 'O'; '.'; 'O'; '.'; 'O'; 'O'; '#'|];
    [|'#'; '.'; '.'; '.'; '.'; 'O'; '.'; '.'; '.'; '#'|];
    [|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|]|];
 robot = (4, 4);
 moves =
  ['<'; 'v'; 'v'; '>'; '^'; '<'; 'v'; '^'; '>'; 'v'; '>'; '^'; 'v'; 'v'; '^';
   'v'; '>'; 'v'; '<'; '>'; 'v'; '^'; 'v'; '<'; 'v'; '<'; '^'; 'v'; 'v'; '<';
   '<'; '<'; '^'; '>'; '<'; '<'; '>'; '<'; '>'; '>'; 'v'; '<'; 'v'; 'v'; 'v';
   '<'; '>'; '^'; 'v'; '^'; '>'; '^'; '<'; '<'; '<'; '>'; '<'; '<'; 'v'; '<';
   '<'; '<'; 'v'; '^'; 'v'; 'v'; '^'; 'v'; '>'; '^'; 'v'; 

That seems plausible.  Now... we must play Sokoban, I guess.

In [183]:
let do_sokoban_move move next_moves sokoban =
  let (di, dj) = match move with
  | '^' -> (-1, 0)
  | '>' -> (0, 1)
  | 'v' -> (1, 0)
  | '<' -> (0, -1)
  | _ -> failwith "bad move" in
  let (i, j) = sokoban.robot in
  let (ni, nj) = (i + di, j + dj) in
  let () = (* push boxes *)
    let p = ref (ni, nj) in
    while Char.(sokoban.grid.(fst !p).(snd !p) = 'O') do
      p := (fst !p + di, snd !p + dj)
    done;
    if Char.(sokoban.grid.(fst !p).(snd !p) = '.') then begin
      sokoban.grid.(fst !p).(snd !p) <- sokoban.grid.(ni).(nj);
      sokoban.grid.(ni).(nj) <- '.';
    end; in
  let there = sokoban.grid.(ni).(nj) in
  match there with
  | '#' | 'O' -> {grid=sokoban.grid; robot=(i, j); moves=next_moves}
  | '.' -> {grid=sokoban.grid; robot=(ni, nj); moves=next_moves}
  | _ -> failwith ("bad grid " ^ (String.make 1 there))

val do_sokoban_move : char -> char list -> sokoban -> sokoban = <fun>


In [184]:
let rec play_sokoban sokoban =
  match sokoban.moves with
  | [] -> sokoban
  | move :: next_moves -> play_sokoban (do_sokoban_move move next_moves sokoban)

val play_sokoban : sokoban -> sokoban = <fun>


In [185]:
read_sokoban "day15_small.txt"
|> do_sokoban_move '<' []
|> do_sokoban_move '^' []
|> do_sokoban_move '^' []
|> do_sokoban_move '>' []
|> do_sokoban_move '>' []

- : sokoban =
{grid =
  [|[|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|];
    [|'#'; '.'; '.'; '.'; '.'; 'O'; 'O'; '#'|];
    [|'#'; '#'; '.'; '.'; 'O'; '.'; '.'; '#'|];
    [|'#'; '.'; '.'; '.'; 'O'; '.'; '.'; '#'|];
    [|'#'; '.'; '#'; '.'; 'O'; '.'; '.'; '#'|];
    [|'#'; '.'; '.'; '.'; 'O'; '.'; '.'; '#'|];
    [|'#'; '.'; '.'; '.'; '.'; '.'; '.'; '#'|];
    [|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|]|];
 robot = (1, 4); moves = []}


In [186]:
read_sokoban "day15_small.txt" |> play_sokoban

- : sokoban =
{grid =
  [|[|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|];
    [|'#'; '.'; '.'; '.'; '.'; 'O'; 'O'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '#'|];
    [|'#'; '.'; '.'; '.'; '.'; '.'; 'O'; '#'|];
    [|'#'; '.'; '#'; 'O'; '.'; '.'; '.'; '#'|];
    [|'#'; '.'; '.'; '.'; 'O'; '.'; '.'; '#'|];
    [|'#'; '.'; '.'; '.'; 'O'; '.'; '.'; '#'|];
    [|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|]|];
 robot = (4, 4); moves = []}


In [187]:
read_sokoban "day15_example.txt" |> play_sokoban

- : sokoban =
{grid =
  [|[|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|];
    [|'#'; '.'; 'O'; '.'; 'O'; '.'; 'O'; 'O'; 'O'; '#'|];
    [|'#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'|];
    [|'#'; 'O'; 'O'; '.'; '.'; '.'; '.'; '.'; '.'; '#'|];
    [|'#'; 'O'; 'O'; '.'; '.'; '.'; '.'; '.'; '.'; '#'|];
    [|'#'; 'O'; '#'; '.'; '.'; '.'; '.'; '.'; 'O'; '#'|];
    [|'#'; 'O'; '.'; '.'; '.'; '.'; '.'; 'O'; 'O'; '#'|];
    [|'#'; 'O'; '.'; '.'; '.'; '.'; '.'; 'O'; 'O'; '#'|];
    [|'#'; 'O'; 'O'; '.'; '.'; '.'; '.'; 'O'; 'O'; '#'|];
    [|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|]|];
 robot = (4, 3); moves = []}


In [188]:
let checksum_boxes sokoban =
  let rows = Array.length sokoban.grid in
  let cols = Array.length sokoban.grid.(0) in
  let sum = ref 0 in
  for i=0 to rows-1 do
    for j=0 to cols-1 do
      if Char.(sokoban.grid.(i).(j) = 'O' || sokoban.grid.(i).(j) = '[') then
        sum := !sum + 100 * i + j 
    done
  done; !sum

val checksum_boxes : sokoban -> int = <fun>


In [189]:
read_sokoban "day15_example.txt" |> play_sokoban |> checksum_boxes

- : int = 10092


In [190]:
read_sokoban "day15_input.txt" |> play_sokoban |> checksum_boxes

- : int = 1415498


### Part 2

Now we get to play Big Sokoban!  It still probably makes sense to do things with grids, but the logic is somewhat more complicated for vertical moves.

In [191]:
let inflate sokoban =
  let size = Array.length sokoban.grid in
  let new_grid = Array.make_matrix size (2 * size) '.' in
  let () =
    for i = 0 to size-1 do
      for j = 0 to size-1 do
        match sokoban.grid.(i).(j) with
        | '#' | '.' ->
          new_grid.(i).(2 * j) <- sokoban.grid.(i).(j);
          new_grid.(i).(2 * j + 1) <- sokoban.grid.(i).(j)
        | 'O' ->
          new_grid.(i).(2 * j) <- '[';
          new_grid.(i).(2 * j + 1) <- ']'
        | _ -> failwith "bad grid"
      done
    done in
  let (i, j) = sokoban.robot in
  {grid=new_grid; robot=(i, 2*j); moves=sokoban.moves}

val inflate : sokoban -> sokoban = <fun>


In [192]:
read_sokoban "day15_small2.txt" |> inflate

- : sokoban =
{grid =
  [|[|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '#'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '['; ']'; '['; ']'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '['; ']'; '.'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|]|];
 robot = (3, 10);
 moves = ['<'; 'v'; 'v'; '<'; '<'; '^'; '^'; '<'; '<'; '^'; '^']}


In Big Sokoban, we can push an arbitrary number of columns.  For example, say we push up in this grid. 
```
  12345678
1 ........
2 [][][][]
3 .[][][].
4 .#[][]#.
5 ..#[]#..
6 ....@...
```

All the boxes should shift up by one row.  So, to push up, we scan up from the current row checking a set of columns.  If we see `]`, we add `column-1` to the set; if we see `[` we add `column+1`.  If we see '#', the move fails, and if we see '.' a column is ok.

I guess we could write the program that way, but this seems a bit lame.

A different way to think about this is that a push (from, to) either succeeds, or _depends on_ other pushes.  If all pushes in the tree succeed, we execute them in depth first order.

In [193]:
type push = {
  from_pos: int * int;
  to_pos: int * int;
  children: push list
}

type push = {
  from_pos : int * int;
  to_pos : int * int;
  children : push list;
}


When checking pushes, we might visit dependent pushes multiple times.  For example, in this situation:
```
..
[] ab
[] cd
.@
```

We will first check d, then b and a. Then we'll check c, which also depends on a and b. We'll just keep track of which pushes we've already checked and construct a tree like $c \rightarrow (d \rightarrow (a \rightarrow b))$.  Because we short-circuit as soon as any push fails, already checked pushes must be valid - we just don't want to add them to the tree again.

In [194]:
type push_check = Bad | AlreadyChecked | Push of push

type push_check = Bad | AlreadyChecked | Push of push


In [195]:
let make_push (fi, fj) (di, dj) grid =
  let already_checked = Hash_set.Poly.of_list [(-1, -1)] in
  let rec make_push' (fi, fj) (di, dj) add_adjacent =
    let check_push (fi, fj) (di, dj) add_adjacent =
      if Hash_set.mem already_checked (fi, fj) then
        AlreadyChecked
      else begin
        (* we are only really checking vertical pushes if add_adjacent=false *)
        if not add_adjacent then Hash_set.add already_checked (fi, fj);
        make_push' (fi, fj) (di, dj) add_adjacent
      end in
    let (ti, tj) = (fi + di, fj + dj) in
    if di = 0 || (not add_adjacent) then
      (* horizontal push, or vertical push w/o adjacent push *)
      match grid.(ti).(tj) with
      | '#' -> Bad
      | '.' -> Push {from_pos=(fi, fj); to_pos=(ti, tj); children=[]}
      | '[' | ']' | 'O' ->
        let child = check_push (ti, tj) (di, dj) true in
          (match child with
           | Bad -> Bad
           | AlreadyChecked -> Push {from_pos=(fi, fj); to_pos=(ti, tj); children=[]}
           | Push p -> Push {from_pos=(fi, fj); to_pos=(ti, tj); children=[p]})
      | _ -> failwith "bad grid"
    else
      (* vertical push with adjacent push *)
      (* first push this column, then dependent on that, push neighbor *)
      match check_push (fi, fj) (di, dj) false with
      | Bad -> Bad
      | AlreadyChecked -> AlreadyChecked
      | Push p ->
        let push_neighbor neighbor =
          match check_push neighbor (di, dj) false with
          | Bad -> Bad
          | AlreadyChecked -> Push p
          | Push n -> Push {from_pos=n.from_pos; to_pos=n.to_pos; children=p::n.children} in 
        match grid.(fi).(fj) with
        | ']' -> push_neighbor (fi, fj-1)
        | '[' -> push_neighbor (fi, fj+1)
        | 'O' -> Push p
        | _ -> failwith "bad grid" in
  match grid.(fi).(fj) with
  | '.' | '#' -> None
  | _ ->
    match make_push' (fi, fj) (di, dj) true with
    | Bad -> None
    | AlreadyChecked -> failwith "bug"
    | Push p -> Some p

val make_push :
  int * int ->
  Core_kernel.Int.t * int -> char Core.Array.t Core.Array.t -> push option =
  <fun>


In [196]:
read_sokoban "day15_small2.txt" |> inflate

- : sokoban =
{grid =
  [|[|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '#'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '['; ']'; '['; ']'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '['; ']'; '.'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|]|];
 robot = (3, 10);
 moves = ['<'; 'v'; 'v'; '<'; '<'; '^'; '^'; '<'; '<'; '^'; '^']}


In [197]:
read_sokoban "day15_small2.txt" |> inflate |> (fun s -> s.grid) |> make_push (4, 7) (-1, 0)

- : push option =
Some
 {from_pos = (4, 6); to_pos = (3, 6);
  children =
   [{from_pos = (4, 7); to_pos = (3, 7);
     children =
      [{from_pos = (3, 6); to_pos = (2, 6);
        children = [{from_pos = (3, 7); to_pos = (2, 7); children = []}]}]}]}


In [198]:
let rec do_push p grid =
  List.iter p.children (fun p' -> do_push p' grid);
  grid.(fst p.to_pos).(snd p.to_pos) <- grid.(fst p.from_pos).(snd p.from_pos); 
  grid.(fst p.from_pos).(snd p.from_pos) <- '.'

val do_push : push -> char Core.Array.t Core.Array.t -> unit = <fun>


In [199]:
let do_sokoban_move move next_moves sokoban =
  let (di, dj) = match move with
  | '^' -> (-1, 0)
  | '>' -> (0, 1)
  | 'v' -> (1, 0)
  | '<' -> (0, -1)
  | _ -> failwith "bad move" in
  let (i, j) = sokoban.robot in
  let (ni, nj) = (i + di, j + dj) in
  let () =
    match (make_push (ni, nj) (di, dj) sokoban.grid) with
    | Some p -> do_push p sokoban.grid
    | None -> () in
  let there = sokoban.grid.(ni).(nj) in
  match there with
  | '#' | '[' | ']' | 'O' ->
    {grid=sokoban.grid; robot=(i, j); moves=next_moves}
  | '.' ->
    {grid=sokoban.grid; robot=(ni, nj); moves=next_moves}
  | _ -> failwith ("bad grid " ^ (String.make 1 there))

val do_sokoban_move : char -> char list -> sokoban -> sokoban = <fun>


In [200]:
read_sokoban "day15_small2.txt" |> inflate
|> do_sokoban_move '<' []
|> do_sokoban_move 'v' []
|> do_sokoban_move 'v' []
|> do_sokoban_move '<' []
|> do_sokoban_move '<' []
|> do_sokoban_move '^' []
|> do_sokoban_move '<' []
|> do_sokoban_move '<' []
|> do_sokoban_move '^' []
|> do_sokoban_move '^' []

- : sokoban =
{grid =
  [|[|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '['; ']'; '.'; '#'; '#'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '['; ']'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '['; ']'; '.'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|]|];
 robot = (2, 5); moves = []}


In [201]:
let rec play_sokoban sokoban =
  match sokoban.moves with
  | [] -> sokoban
  | move :: next_moves -> play_sokoban (do_sokoban_move move next_moves sokoban)

val play_sokoban : sokoban -> sokoban = <fun>


In [202]:
read_sokoban "day15_small2.txt" |> inflate |> play_sokoban

- : sokoban =
{grid =
  [|[|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '['; ']'; '.'; '#'; '#'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '['; ']'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '['; ']'; '.'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'|]|];
 robot = (2, 5); moves = []}


In [203]:
read_sokoban "day15_example.txt" |> inflate |> play_sokoban

- : sokoban =
{grid =
  [|[|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#';
      '#'; '#'; '#'; '#'; '#'; '#'|];
    [|'#'; '#'; '['; ']'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '['; ']'; '.';
      '['; ']'; '['; ']'; '#'; '#'|];
    [|'#'; '#'; '['; ']'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.';
      '.'; '['; ']'; '.'; '#'; '#'|];
    [|'#'; '#'; '['; ']'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '['; ']';
      '['; ']'; '['; ']'; '#'; '#'|];
    [|'#'; '#'; '['; ']'; '.'; '.'; '.'; '.'; '.'; '.'; '['; ']'; '.'; '.';
      '.'; '.'; '['; ']'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '#'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '['; ']';
      '.'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '['; ']'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.';
      '.'; '.'; '.'; '.'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '['; ']'; '.';
      '['; ']'; '['; ']'; '#'; '#'|];
    [|'#'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '['; ']'; '['; ']'; 

In [204]:
read_sokoban "day15_example.txt" |> inflate |> play_sokoban |> checksum_boxes

- : int = 9021


In [205]:
read_sokoban "day15_input.txt" |> inflate |> play_sokoban |> checksum_boxes

- : int = 1432898


## <a name="day16">Day 16</a>

### Part 1

Another day, another grid.  Today we have to search for the lowest cost path through a maze.

In [206]:
type day16_maze = {
  grid: char Core.Array.t Core.Array.t;
  start: int * int;
  goal: int * int;
}

type day16_maze = {
  grid : char Core.Array.t Core.Array.t;
  start : int * int;
  goal : int * int;
}


In [207]:
let read_maze filename =
  let grid = read_grid_from_file filename in
  let start = find_in_grid (Char.(=) 'S') grid in
  let goal = find_in_grid (Char.(=) 'E') grid in
  {grid; start; goal}

val read_maze : Base.string -> day16_maze = <fun>


In [208]:
read_maze "day16_example.txt"

- : day16_maze =
{grid =
  [|[|'#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#'; '#';
      '#'|];
    [|'#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '.'; '.'; '.'; '.'; 'E';
      '#'|];
    [|'#'; '.'; '#'; '.'; '#'; '#'; '#'; '.'; '#'; '.'; '#'; '#'; '#'; '.';
      '#'|];
    [|'#'; '.'; '.'; '.'; '.'; '.'; '#'; '.'; '#'; '.'; '.'; '.'; '#'; '.';
      '#'|];
    [|'#'; '.'; '#'; '#'; '#'; '.'; '#'; '#'; '#'; '#'; '#'; '.'; '#'; '.';
      '#'|];
    [|'#'; '.'; '#'; '.'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '.';
      '#'|];
    [|'#'; '.'; '#'; '.'; '#'; '#'; '#'; '#'; '#'; '.'; '#'; '#'; '#'; '.';
      '#'|];
    [|'#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '.';
      '#'|];
    [|'#'; '#'; '#'; '.'; '#'; '.'; '#'; '#'; '#'; '#'; '#'; '.'; '#'; '.';
      '#'|];
    [|'#'; '.'; '.'; '.'; '#'; '.'; '.'; '.'; '.'; '.'; '#'; '.'; '#'; '.';
      '#'|];
    [|'#'; '.'; '#'; '.'; '#'; '.'; '#'; '#'; '#'; '.'; '#'; '.'; '#'; '.';
      '#'

All right... do we have a heap of some sort?  There is `core_kernel.Pairing_heap`, but I'm not sure that `core_kernel` exists anymore.  According to a blog post, it was going to stop existing, but I don't know if that's happened or not in my version of ocaml.

In [209]:
#require "core_kernel" ;;
#require "core_kernel.pairing_heap" ;;

/Users/jered/.opam/4.12.0/lib/core_kernel/tuple_pool: added to search path
/Users/jered/.opam/4.12.0/lib/core_kernel/tuple_pool/tuple_pool.cma: loaded
/Users/jered/.opam/4.12.0/lib/core_kernel/pairing_heap: added to search path
/Users/jered/.opam/4.12.0/lib/core_kernel/pairing_heap/pairing_heap.cma: loaded


In [210]:
Pairing_heap.create

- : ?min_size:int -> cmp:('a -> 'a -> int) -> unit -> 'a Pairing_heap.t =
<fun>


It's a Festivus miracle!

In [211]:
type day16_position = {
  pos: int * int;
  facing: int * int;
}

type day16_position = { pos : int * int; facing : int * int; }


In [212]:
let search_maze maze =
  let size = Array.length maze.grid in
  (* 0 1 -> 1 0 -> 0 -1 -> -1 0 *)
  let turn_cw dir = (snd dir, - fst dir) in
  (* 0 1 -> -1 0 -> 0 -1 -> 1 0 *)
  let turn_ccw dir = (- snd dir, fst dir) in
  let compare_paths (p_cost, _) (q_cost, _) = Stdlib.compare p_cost q_cost in
  let start = {pos=maze.start; facing=(0, 1)} in
  let queue = Pairing_heap.of_list ~cmp:compare_paths [(0, start)] in
  let visited = Hash_set.Poly.of_list [start] in
  let enqueue cost p =
    if not (Hash_set.mem visited p) then begin
      Pairing_heap.add queue (cost, p)
    end in
  let best_cost = ref (-1) in
  while !best_cost < 0 && not (Pairing_heap.is_empty queue) do 
    let (cost, p) = Pairing_heap.pop_exn queue in
    Hash_set.add visited p;
    if Poly.(=) p.pos maze.goal then
      best_cost := cost
    else begin
      let (i, j) = p.pos in
      let (di, dj) = p.facing in
      let (ni, nj) = (i + di, j + dj) in
      if ni >= 0 && ni < size && nj >= 0 && nj < size && Char.(maze.grid.(ni).(nj) <> '#') then
        enqueue (cost + 1) {pos=(ni, nj); facing=p.facing};
      enqueue (cost + 1000) {pos=p.pos; facing=turn_cw p.facing};
      enqueue (cost + 1000) {pos=p.pos; facing=turn_ccw p.facing};
    end
  done; !best_cost

val search_maze : day16_maze -> int = <fun>


In [213]:
read_maze "day16_example.txt" |> search_maze

- : int = 7036


In [214]:
read_maze "day16_input.txt" |> search_maze

- : int = 78428


### Part 2

Now we have to count cells in best paths.

In [215]:
let count_best_cells maze =
  let size = Array.length maze.grid in
  let turn_cw dir = (snd dir, - fst dir) in
  let turn_ccw dir = (- snd dir, fst dir) in
  let compare_paths (p_cost, _, _) (q_cost, _, _) = Stdlib.compare p_cost q_cost in
  let start = {pos=maze.start; facing=(0, 1)} in
  let queue = Pairing_heap.of_list ~cmp:compare_paths [(0, start, [maze.start])] in
  let best_cells = Hash_set.Poly.of_list [maze.start] in
  let visited = Hash_set.Poly.of_list [{pos=maze.start; facing=(0, 1)}] in
  let enqueue cost p history =
    if not (Hash_set.mem visited p) then begin
      Pairing_heap.add queue (cost, p, p.pos::history)
    end in
  let is_done = ref false in
  let best_cost = ref (-1) in
  while not !is_done && not (Pairing_heap.is_empty queue) do 
    let (cost, p, history) = Pairing_heap.pop_exn queue in
    Hash_set.add visited p;
    if !best_cost > 0 && cost > !best_cost then
      is_done := true
    else if Poly.(=) p.pos maze.goal then begin
      best_cost := cost;
      List.iter history (Hash_set.add best_cells)
    end else begin
      let (i, j) = p.pos in
      let (di, dj) = p.facing in
      let (ni, nj) = (i + di, j + dj) in
      if ni >= 0 && ni < size && nj >= 0 && nj < size && Char.(maze.grid.(ni).(nj) <> '#') then
        enqueue (cost + 1) {pos=(ni, nj); facing=p.facing} history;
      enqueue (cost + 1000) {pos=p.pos; facing=turn_cw p.facing} history;
      enqueue (cost + 1000) {pos=p.pos; facing=turn_ccw p.facing} history;
    end
  done;
  Hash_set.length best_cells

val count_best_cells : day16_maze -> int = <fun>


In [216]:
read_maze "day16_example.txt" |> count_best_cells

- : int = 45


In [217]:
read_maze "day16_example2.txt" |> count_best_cells

- : int = 64


In [218]:
read_maze "day16_input.txt" |> count_best_cells

- : int = 463


## Day 17

### Part 1

Today we're emulating a 3-bit computer.  I love emulating computers.

In [219]:
type day17_operand = Literal of int | AReg | BReg | CReg

type day17_operand = Literal of int | AReg | BReg | CReg


In [220]:
type day17_instruction =
| Adv of day17_operand
| Bxl of day17_operand
| Bst of day17_operand
| Jnz of day17_operand
| Bxc of day17_operand
| Out of day17_operand
| Bdv of day17_operand
| Cdv of day17_operand

type day17_instruction =
    Adv of day17_operand
  | Bxl of day17_operand
  | Bst of day17_operand
  | Jnz of day17_operand
  | Bxc of day17_operand
  | Out of day17_operand
  | Bdv of day17_operand
  | Cdv of day17_operand


In [221]:
type day17_computer = {
  mutable a: int;
  mutable b: int;
  mutable c: int;
  mutable ip: int;
  mutable output: int list;
  program: day17_instruction Core.Array.t;
  bytecode: int list;
}

type day17_computer = {
  mutable a : int;
  mutable b : int;
  mutable c : int;
  mutable ip : int;
  mutable output : int list;
  program : day17_instruction Core.Array.t;
  bytecode : int list;
}


In [222]:
let disassemble program =
  let parse_combo operand =
    match operand with
    | 0 | 1 | 2 | 3 -> Literal operand
    | 4 -> AReg
    | 5 -> BReg
    | 6 -> CReg
    | _ -> failwith "bad operand" in
  let parse opcode operand =
    match opcode with
    | 0 -> Adv (parse_combo operand)
    | 1 -> Bxl (Literal operand)
    | 2 -> Bst (parse_combo operand)
    | 3 -> Jnz (Literal operand)
    | 4 -> Bxc (Literal operand)
    | 5 -> Out (parse_combo operand)
    | 6 -> Bdv (parse_combo operand)
    | 7 -> Cdv (parse_combo operand)
    | _ -> failwith "bad opcode" in
  let opcodes = filteri program ~f:(fun i _ -> i mod 2 = 0) in
  let operands = filteri program ~f:(fun i _ -> i mod 2 = 1) in
  map2_exn opcodes operands parse

val disassemble : int list -> day17_instruction list = <fun>


In [223]:
let read_day17_computer filename =
  let lines = In_channel.read_lines filename in
  let parse_register line =
    String.split line ~on:':' |> last_exn |> String.strip |> int_of_string in
  let (a, b, c) = match take lines 3 >>| parse_register with
  | [a; b; c] -> (a, b, c)
  | _ -> failwith "missing regs" in
  let parse_program line =
    String.split line ~on:' ' |> last_exn |> String.split ~on:',' >>| int_of_string in
  let bytecode = parse_program (last_exn lines) in
  let program = Array.of_list @@ disassemble bytecode in
  {a; b; c; program; bytecode; ip=0; output=[]}

val read_day17_computer : Base.string -> day17_computer = <fun>


In [224]:
read_day17_computer "day17_example.txt"

- : day17_computer =
{a = 729; b = 0; c = 0; ip = 0; output = [];
 program = [|Adv (Literal 1); Out AReg; Jnz (Literal 0)|];
 bytecode = [0; 1; 5; 4; 3; 0]}


All right, now we can run the programs...

In [225]:
let step_computer state =
  let decode operand =
    match operand with
    | Literal n -> n
    | AReg -> state.a
    | BReg -> state.b
    | CReg -> state.c in
  let execute f = begin f; state.ip <- state.ip + 2 end in
  let () = match state.program.(state.ip / 2) with
  | Adv operand ->
    execute (state.a <- state.a / (Int.shift_left 1 (decode operand)))
  | Bxl operand ->
    execute (state.b <- state.b lxor (decode operand))
  | Bst operand ->
    execute (state.b <- (decode operand) land 7)
  | Jnz operand ->
    if state.a <> 0 then
      state.ip <- decode operand
    else
      state.ip <- state.ip + 2
  | Bxc _ ->
    execute (state.b <- state.b lxor state.c)
  | Out operand ->
    execute (state.output <- ((decode operand) land 7)::state.output)
  | Bdv operand ->
    execute (state.b <- state.a / (Int.shift_left 1 (decode operand)))
  | Cdv operand ->
    execute (state.c <- state.a / (Int.shift_left 1 (decode operand))) in
  state

val step_computer : day17_computer -> day17_computer = <fun>


In [226]:
read_day17_computer "day17_example.txt" |> iterate 4 step_computer

- : day17_computer =
{a = 182; b = 0; c = 0; ip = 2; output = [4];
 program = [|Adv (Literal 1); Out AReg; Jnz (Literal 0)|];
 bytecode = [0; 1; 5; 4; 3; 0]}


In [227]:
let emulate state =
  let size = Array.length state.program in
  while state.ip >= 0 && state.ip < 2*size do
    let _ = step_computer state in ()
  done; rev state.output

val emulate : day17_computer -> int list = <fun>


In [228]:
read_day17_computer "day17_example.txt" |> emulate >>| string_of_int |> String.concat ~sep:","

- : string = "4,6,3,5,6,3,5,2,1,0"


In [229]:
read_day17_computer "day17_input.txt" |> emulate >>| string_of_int |> String.concat ~sep:","

- : string = "3,1,4,3,1,7,1,6,3"


### Part 2

We have to find an initial value for the A register that makes the program function as a [quine](https://en.wikipedia.org/wiki/Quine_(computing)) - that's pretty cool actually!

In [230]:
read_day17_computer "day17_quine.txt" |> fun s -> s.a <- 117440; s |> emulate

- : int list = [0; 3; 5; 4; 3; 0]


Ok, what is the input program actually doing?

In [231]:
read_day17_computer "day17_input.txt"

- : day17_computer =
{a = 64751475; b = 0; c = 0; ip = 0; output = [];
 program =
  [|Bst AReg; Bxl (Literal 2); Cdv BReg; Bxc (Literal 5); Bxl (Literal 3);
    Out BReg; Adv (Literal 3); Jnz (Literal 0)|];
 bytecode = [2; 4; 1; 2; 7; 5; 4; 5; 1; 3; 5; 5; 0; 3; 3; 0]}


A loop examines 3 bits of A at a time, and outputs once per iteration.  So A must have 48 bits to get the right length output...

In [232]:
read_day17_computer "day17_input.txt" |> fun s -> s.a <- (Int.shift_left 1 47); s |> emulate

- : int list = [1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 5]


In pseudo-code, the program is...
```
while True:
  b = (a & 7) ^ 2
  c = a / (1 << b)
  b = (b ^ c) ^ 3
  output(b & 7)
  a = a >> 3
  if a = 0: break
```

The output for each 3-bit group depends on that group, and on some group to the left.  But the _leftmost_ group - i.e. the last output - can only depend on itself.  So, we should be able to construct the bit string from left to right.

Initially I tried taking the smallest group at each index, but that fails - there can still be a few options at each digit position so we still have to search.  But there are not many possibilities, and it is quick.

In [233]:
let find_quine state =
  let copy state =
    {a=state.a; b=state.b; c=state.c; ip=state.ip;
     program=state.program; bytecode=state.bytecode;
     output=state.output} in
  let size = List.length state.bytecode in
  let rec search input_state acc i g =
    if i = (-1) then
      Some acc
    else if g = 8 then
      None
    else begin
      let trial = acc + (Int.shift_left g (3 * i)) in
      let state = copy input_state in
      let () = state.a <- trial in
      let _ = emulate state in
      let ith_output = List.nth_exn state.output ((size - 1) - i) in
      let ith_bytecode = List.nth_exn state.bytecode i in
      if ith_output = ith_bytecode then
        match search input_state trial (i - 1) 0 with
        | Some result -> Some result
        | None -> search input_state acc i (g + 1)
      else search input_state acc i (g + 1)
    end in
  search state 0 (size - 1) 1

val find_quine : day17_computer -> int option = <fun>


In [234]:
read_day17_computer "day17_input.txt" |> find_quine

- : int option = Some 37221270076916


In [235]:
read_day17_computer "day17_input.txt" |> fun s -> s.a <- 37221270076916; s |> emulate

- : int list = [2; 4; 1; 2; 7; 5; 4; 5; 1; 3; 5; 5; 0; 3; 3; 0]


## Day 18

### Part 1

Today we have to find a path through a grid _but there is no grid_.

In [236]:
let read_points filename =
  let parse line =
    String.split line ~on:',' >>| int_of_string |> pair_of_list in
  In_channel.read_lines filename >>| parse

val read_points : Base.string -> (int * int) list = <fun>


In [237]:
read_points "day18_example.txt"

- : (int * int) list =
[(5, 4); (4, 2); (4, 5); (3, 0); (2, 1); (6, 3); (2, 4); (1, 5); (0, 6);
 (3, 3); (2, 6); (5, 1); (1, 2); (5, 5); (2, 5); (6, 5); (1, 4); (0, 4);
 (6, 4); (1, 1); (6, 1); (1, 0); (0, 5); (1, 6); (2, 0)]


Didn't I already write a best-first search thing?  Yeah, let's copy and paste that and tweak it.

Hmm... there seem to be lots of paths here, I might need A*.

In [238]:
let find_escape_route start goal size walls =
  let wall_set = Hash_set.Poly.of_list walls in
  let compare_paths (p_cost, _, _) (q_cost, _, _) = Stdlib.compare p_cost q_cost in
  let estimate_cost steps (i, j) =
    let (gi, gj) = goal in steps + abs(i - gi) + abs(j - gj) in
  let queue = Pairing_heap.of_list ~cmp:compare_paths [(estimate_cost 0 start, 0, start)] in
  let visited = Hash_set.Poly.of_list [start] in
  let enqueue steps p =
    if not (Hash_set.mem visited p) then begin
      Pairing_heap.add queue (estimate_cost steps p, steps, p)
    end in
  let best_steps = ref (-1) in
  while !best_steps < 0 && not (Pairing_heap.is_empty queue) do 
    let (_, steps, p) = Pairing_heap.pop_exn queue in
    Hash_set.add visited p;
    if Poly.(=) p goal then
      best_steps := steps
    else begin
      let in_bounds (i, j) = i >= 0 && i <= size && j >= 0 && j <= size in
      let (i, j) = p in
      let neighbors = [(i + 1, j); (i - 1, j); (i, j - 1); (i, j + 1)] in
      let check n =
        if in_bounds n && (not @@ Hash_set.mem wall_set n) then
          enqueue (steps + 1) n in
      List.iter neighbors check
    end
  done; !best_steps

val find_escape_route :
  Core_kernel.Int.t * Core_kernel.Int.t ->
  Core_kernel.Int.t * Core_kernel.Int.t ->
  Core_kernel.Int.t ->
  (Core_kernel.Int.t * Core_kernel.Int.t) Core.Hash_set.Poly.elt list -> int =
  <fun>


In [239]:
take (read_points "day18_example.txt") 12 |> find_escape_route (0, 0) (6, 6) 6

- : int = 22


In [240]:
take (read_points "day18_input.txt") 1024 |> find_escape_route (0, 0) (70, 70) 70

- : int = 382


### Part 2

Now we are solving a different problem - we just need to know if there is any path through the point cloud, there is no need to find the best.

In [241]:
let can_escape start goal size walls =
  let wall_set = Hash_set.Poly.of_list walls in
  let queued = Hash_set.Poly.of_list [start] in
  let rec search p =
    if Poly.(=) p goal then
      true
    else
      let in_bounds (i, j) = i >= 0 && i <= size && j >= 0 && j <= size in
      let not_a_wall n = not @@ Hash_set.mem wall_set n in
      let not_queued n = not @@ Hash_set.mem queued n in
      let (i, j) = p in
      let neighbors = [(i + 1, j); (i - 1, j); (i, j - 1); (i, j + 1)] in
      let search_neighbor n =
        if in_bounds n && not_a_wall n && not_queued n then begin
          Hash_set.add queued n;
          search n
        end else false in
      List.exists neighbors search_neighbor in
  search start

val can_escape :
  (Core_kernel.Int.t * Core_kernel.Int.t) Core.Hash_set.Poly.elt ->
  Core_kernel.Int.t * Core_kernel.Int.t ->
  Core_kernel.Int.t ->
  (Core_kernel.Int.t * Core_kernel.Int.t) Core.Hash_set.Poly.elt list -> bool =
  <fun>


In [242]:
take (read_points "day18_example.txt") 20 |> can_escape (0, 0) (6, 6) 6

- : bool = true


In [243]:
take (read_points "day18_example.txt") 21 |> can_escape (0, 0) (6, 6) 6

- : bool = false


We could binary search if we must, but I'm going to try just using a for loop first.

In [244]:
let find_blocker start goal size walls =
  let num_walls = List.length walls in
  let j = ref 0 in
  for i=0 to num_walls-1 do
    if !j = 0 && not @@ can_escape start goal size (take walls (i + 1)) then
      j := i
  done; List.nth_exn walls !j

val find_blocker :
  (Core_kernel.Int.t * Core_kernel.Int.t) Core.Hash_set.Poly.elt ->
  Core_kernel.Int.t * Core_kernel.Int.t ->
  Core_kernel.Int.t ->
  (Core_kernel.Int.t * Core_kernel.Int.t) Core.Hash_set.Poly.elt list ->
  (Core_kernel.Int.t * Core_kernel.Int.t) Core.Hash_set.Poly.elt = <fun>


In [245]:
read_points "day18_example.txt" |> find_blocker (0, 0) (6, 6) 6

- : (Core_kernel.Int.t * Core_kernel.Int.t) Core.Hash_set.Poly.elt = (6, 1)


In [246]:
(*read_points "day18_input.txt" |> find_blocker (0, 0) (70, 70) 70 *)

That worked, but it was a little slow.  Let's write a janky busted binary search that should work well enough for this problem, but is otherwise riddled with bugs.

In [247]:
let find_blocker start goal size walls =
  let num_walls = List.length walls in
  let rec search lower upper =
    if upper < lower then lower
    else let mid = (lower + upper) / 2 in
      match can_escape start goal size (take walls (mid + 1)) with
      | false -> search lower (mid - 1)
      | true -> search (mid + 1) upper in
  List.nth_exn walls (search 0 num_walls)

val find_blocker :
  (Core_kernel.Int.t * Core_kernel.Int.t) Core.Hash_set.Poly.elt ->
  Core_kernel.Int.t * Core_kernel.Int.t ->
  Core_kernel.Int.t ->
  (Core_kernel.Int.t * Core_kernel.Int.t) Core.Hash_set.Poly.elt list ->
  (Core_kernel.Int.t * Core_kernel.Int.t) Core.Hash_set.Poly.elt = <fun>


In [248]:
read_points "day18_example.txt" |> find_blocker (0, 0) (6, 6) 6

- : (Core_kernel.Int.t * Core_kernel.Int.t) Core.Hash_set.Poly.elt = (6, 1)


In [249]:
read_points "day18_input.txt" |> find_blocker (0, 0) (70, 70) 70

- : (Core_kernel.Int.t * Core_kernel.Int.t) Core.Hash_set.Poly.elt = (6, 36)


## Day 19

### Part 1

We have to see which strings we can make out of some substrings.  I'll just use a regex.

In [250]:
type day19_problem = {
  atoms: string list;
  words: string list;
}

type day19_problem = { atoms : string list; words : string list; }


In [257]:
let read_day19_problem filename =
  let lines = In_channel.read_lines filename in
  let parse_atoms line = String.split line ~on:','
    >>| String.strip ~drop:(fun c -> Char.(c = ' ')) in
  let atoms = parse_atoms (hd_exn lines) in
  let words = List.drop lines 2 in
  {atoms; words}

val read_day19_problem : Base.string -> day19_problem = <fun>


In [258]:
read_day19_problem "day19_example.txt"

- : day19_problem =
{atoms = ["r"; "wr"; "b"; "g"; "bwu"; "rb"; "gb"; "br"];
 words =
  ["brwrr"; "bggr"; "gbbr"; "rrbgbr"; "ubwu"; "bwurrg"; "brgr"; "bbrgwb"]}


In [263]:
let day19_part1 problem =
  let can_make word =
    let pattern = "^(" ^ (String.concat ~sep:"|" problem.atoms) ^ ")+$" in
    let re = Re2.create_exn pattern in
    Re2.matches re word in
  problem.words |> List.count ~f:can_make

val day19_part1 : day19_problem -> int = <fun>


In [264]:
read_day19_problem "day19_example.txt" |> day19_part1

- : int = 6


In [265]:
read_day19_problem "day19_input.txt" |> day19_part1

- : int = 358


### Part 2

Now we have to count the ways we can make a string out of substrings.  Let's try writing this as a natural recursion and then memoize.

In [277]:
let count_substring_arrangements problem =
  let sum = fold ~init:0 ~f:(+) in
  let memo = Hashtbl.Poly.of_alist_exn [("", 1)] in
  let rec count prefixes word =
    match Hashtbl.find memo word with
    | Some cached_result -> cached_result
    | None ->
      let count_with_prefix prefix = 
        match String.chop_prefix word ~prefix:prefix with
        | Some rest -> count prefixes rest
        | None -> 0 in
      let result = prefixes >>| count_with_prefix |> sum in
      Hashtbl.add_exn memo ~key:word ~data:result; result in
  let count_word word = count problem.atoms word in
  problem.words >>| count_word |> sum

val count_substring_arrangements : day19_problem -> int = <fun>


In [278]:
read_day19_problem "day19_example.txt" |> count_substring_arrangements

- : int = 16


In [279]:
read_day19_problem "day19_input.txt" |> count_substring_arrangements

- : int = 600639829400603
